In [1]:
import sys
sys.path.append('C:/Users/gebel/github/football_data_aggr/utils')

import oracle_conn
db = oracle_conn.OracleDB(is_cloud=False) 

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
import glob
import pandas as pd
import numpy as np
import re
import plotly_express as px
import warnings
import json
import streamlit as st
import plotly.graph_objects as go
from pathlib import Path

pd.options.display.float_format = "{:,.1f}".format
warnings.filterwarnings('ignore')

# TEAMNAMES value in teamnamedict must match the htdatan teamname!
global teamnamedict
# C:\Users\mg\JupyterLabDir\Rest\Pr Winning\teamnamedict_streamlit.json
with open('scripts/streamlit_app/teamnamedict_streamlit.json') as f:
    teamnamedict = json.load(f)

global widthfig
widthfig = 700
heightfig = 600

title_x=.2 # alignment of title of plotly diagrams. 0 = left, 1 = right

# get all the gamestatistics from in dropdown specified league and season
# setup the database connection.  There's no need to setup cursors with pandas psql.
# tables = list(glob.glob("data/htdatan/*"))
# tables

In [9]:
df_list_all_tables = db.list_htdatan_tables()
df_list_all_tables

,TABLE_NAME
0,B_2122
1,B_2223
2,B_2324
3,B_2425
4,L1_2122
5,L1_2223
6,L1_2324
7,L1_2425
8,LL_2122
9,LL_2223


In [12]:
# connection = db.get_connection()
df_complete_saison = db.show_table('LL_2425')
df_complete_saison[df_complete_saison["H_TEAMNAMES"]=="Alaves"]

,H_DATETIME,H_GAMEINFO,H_TEAMNAMES,H_GOALS,H_BALL_POSSESSION,H_GOAL_ATTEMPTS,H_SHOTS_ON_GOAL,H_SHOTS_OFF_GOAL,H_BLOCKED_SHOTS,H_FREE_KICKS,H_CORNER_KICKS,H_OFFSIDES,H_GOALKEEPER_SAVES,H_FOULS,H_YELLOW_CARDS,H_RED_CARDS,H_TOTAL_PASSES,H_COMPLETED_PASSES,H_TACKLES,H_ATTACKS,H_DANGEROUS_ATTACKS,A_DATETIME,A_GAMEINFO,A_TEAMNAMES,A_GOALS,A_BALL_POSSESSION,A_GOAL_ATTEMPTS,A_SHOTS_ON_GOAL,A_SHOTS_OFF_GOAL,A_BLOCKED_SHOTS,A_FREE_KICKS,A_CORNER_KICKS,A_OFFSIDES,A_GOALKEEPER_SAVES,A_FOULS,A_YELLOW_CARDS,A_RED_CARDS,A_TOTAL_PASSES,A_COMPLETED_PASSES,A_TACKLES,A_ATTACKS,A_DANGEROUS_ATTACKS,H_CAT,GAMEURL,A_CROSSES,A_BIG_CHANCES,A_HIT_THE_WOODWORK,A_THROWINS,A_PASSES_IN_THE_FINAL_THIRD,H_PASSES_IN_THE_FINAL_THIRD,H_TOUCHES_IN_THE_OPPOSITION_BOX,H_BIG_CHANCES,A_SHOTS_OUTSIDE_THE_BOX,A_SHOTS_INSIDE_THE_BOX,A_TOUCHES_IN_THE_OPPOSITION_BOX,H_CLEARANCES_TOTAL,H_EXPECTED_GOALS_XG,H_THROWINS,H_SHOTS_INSIDE_THE_BOX,A_INTERCEPTIONS,A_CLEARANCES_TOTAL,H_CROSSES,H_HIT_THE_WOODWORK,H_INTERCEPTIONS,H_SHOTS_OUTSIDE_THE_BOX,A_EXPECTED_GOALS_XG,H_HEADED_GOALS,A_HEADED_GOALS,HALFTIME
1,01.11.2024 20:00,FOOTBALL\nSPAIN: LALIGA - ROUND 12,Alaves,0,51,6,0,3,3.0,6,2,0,1,12,3.0,NaN,164.0,122.0,NaN,NaN,NaN,01.11.2024 20:00,FOOTBALL\nSPAIN: LALIGA - ROUND 12,Mallorca,0,49,3,1,0,2.0,12,1,2,0,6,0.0,NaN,None,None,NaN,NaN,NaN,1,https://www.flashscore.com/match/02cHDDPH/#/ma...,NaN,None,None,12,NaN,NaN,None,None,None,None,None,None,0.30,12,None,10,None,NaN,None,6,None,0.19,None,None,1.0
2,01.11.2024 20:00,FOOTBALL\nSPAIN: LALIGA - ROUND 12,Alaves,1,53,7,3,1,3.0,6,5,0,1,7,0.0,NaN,159.0,105.0,NaN,NaN,NaN,01.11.2024 20:00,FOOTBALL\nSPAIN: LALIGA - ROUND 12,Mallorca,0,47,4,1,3,0.0,7,1,1,2,6,2.0,NaN,None,None,NaN,NaN,NaN,2,https://www.flashscore.com/match/02cHDDPH/#/ma...,NaN,None,None,16,NaN,NaN,None,None,None,None,None,None,0.85,11,None,3,None,NaN,None,2,None,0.32,None,None,2.0
21,11.01.2025 13:00,FOOTBALL\nSPAIN: LALIGA - ROUND 19,Alaves,0,43,4,1,3,0.0,7,2,0,0,9,0.0,NaN,179.0,145.0,78.0,NaN,NaN,11.01.2025 13:00,FOOTBALL\nSPAIN: LALIGA - ROUND 19,Girona,0,57,2,0,2,0.0,9,2,1,1,7,2.0,NaN,None,None,43.0,NaN,NaN,1,https://www.flashscore.com/match/drfKcJnn/#/ma...,14.0,0,0,10,73.0,63.0,6,1,1,1,4,9,1.21,6,3,2,7,27.0,0,8,1,0.04,None,None,1.0
22,11.01.2025 13:00,FOOTBALL\nSPAIN: LALIGA - ROUND 19,Alaves,0,43,7,1,4,2.0,4,4,1,0,7,1.0,NaN,166.0,128.0,71.0,NaN,NaN,11.01.2025 13:00,FOOTBALL\nSPAIN: LALIGA - ROUND 19,Girona,1,57,1,1,0,0.0,7,1,0,1,4,2.0,NaN,None,None,60.0,NaN,NaN,2,https://www.flashscore.com/match/drfKcJnn/#/ma...,0.0,1,0,10,75.0,68.0,10,1,0,1,14,15,0.47,10,2,8,21,12.0,0,3,5,0.52,None,None,2.0
60,01.09.2024 17:00,FOOTBALL\r\nSPAIN: LALIGA - ROUND 4,Alaves,1,35,10,4,4,2.0,5,3,0,5,5,1.0,NaN,NaN,NaN,NaN,65.0,22.0,01.09.2024 17:00,FOOTBALL\r\nSPAIN: LALIGA - ROUND 4,Las Palmas,0,65,7,5,1,1.0,6,2,0,3,5,0.0,NaN,None,None,NaN,66.0,17.0,cat70,https://www.flashscore.com/match/tQQLbGv9/#/ma...,NaN,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,NaN
61,01.09.2024 17:00,FOOTBALL\r\nSPAIN: LALIGA - ROUND 4,Alaves,1,23,5,1,2,2.0,4,3,1,2,8,2.0,NaN,NaN,NaN,NaN,31.0,16.0,01.09.2024 17:00,FOOTBALL\r\nSPAIN: LALIGA - ROUND 4,Las Palmas,0,77,5,2,3,0.0,10,3,0,0,4,0.0,NaN,None,None,NaN,62.0,16.0,cat71,https://www.flashscore.com/match/tQQLbGv9/#/ma...,NaN,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,NaN
115,30.11.2024 15:15,FOOTBALL\nSPAIN: LALIGA - ROUND 15,Alaves,0,55,5,2,2,1.0,9,4,1,2,6,0.0,NaN,156.0,119.0,70.0,NaN,NaN,30.11.2024 15:15,FOOTBALL\nSPAIN: LALIGA - ROUND 15,Leganes,0,45,3,2,0,1.0,6,7,2,2,9,1.0,NaN,None,None,83.0,NaN,NaN,1,https://www.flashscore.com/match/WIFl0cjf/#/ma...,22.0,1,0,11,54.0,52.0,13,0,0,3,9,10,0.44,13,4,4,18,40.0,0,1,1,0.23,None,None,1.0
116,30.11.2024 15:15,FOOTBALL\nSPAIN: LALIGA - ROUND 15,Alaves,1,65,13,5,2,6.0,6,9,0,0,9,1.0,NaN,196.0,158.0,50.0,NaN,NaN,30.11.2024 15:15,FOOTBALL\nSPAIN: LALIGA - ROUND 15,Leganes,1,35,3,1,0,2.0,9,1,

In [4]:
# map league shortcuts to real names:
shortcut_league_dict = {
    "ll": "La-Liga",
    "pl": "Premier-League",
    "sa": "Serie-A",
    "pl": "Premier-League",
    "l1": "League-1",
    "b": "Bundesliga",
}

cleaned_names_saissons = [e.replace(
    key, val) for e in cleaned_names_saissons for key, val in shortcut_league_dict.items() if key in e]

# sort list by int substring
cleaned_names_saissons = sorted(cleaned_names_saissons, key=lambda x: int(
    x.split(" ")[-1]), reverse=True)

cleaned_names_saissons

['Bundesliga 2425',
 'League-1 2425',
 'La-Liga 2425',
 'Premier-League 2425',
 'Serie-A 2425',
 'Bundesliga 2324',
 'League-1 2324',
 'La-Liga 2324',
 'Premier-League 2324',
 'Serie-A 2324',
 'Bundesliga 2223',
 'League-1 2223',
 'La-Liga 2223',
 'Premier-League 2223',
 'Serie-A 2223',
 'Bundesliga 2122',
 'League-1 2122',
 'La-Liga 2122',
 'Premier-League 2122',
 'Serie-A 2122']

In [4]:
global saison
saison = st.sidebar.selectbox("Saison", list(
    cleaned_names_saissons), 0)
print(saison)

2025-01-15 21:08:26.263 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-15 21:08:26.265 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-15 21:08:26.265 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-15 21:08:26.265 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-15 21:08:26.268 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-01-15 21:08:26.268 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

b 2122


In [5]:
shortcut_league_dict

NameError: name 'shortcut_league_dict' is not defined

In [6]:
def find_key(input_dict, value):
    for key, val in input_dict.items():
        if val == value:
            return key
    return "None"

saison = "{}_{}".format(find_key(shortcut_league_dict, saison.split(" ")[0]),
                        saison.split(" ")[1])
saison

NameError: name 'shortcut_league_dict' is not defined

In [7]:
connection = db.get_connection()
df_complete_saison = db.show_table(saison)
df_complete_saison

2025-01-15 21:08:30,756 - ERROR - Database connection error: ORA-03047: number '2122' is not syntactically valid following 'SELECT * FROM b '
Help: https://docs.oracle.com/error-help/db/ora-03047/


DatabaseError: ORA-03047: number '2122' is not syntactically valid following 'SELECT * FROM b '
Help: https://docs.oracle.com/error-help/db/ora-03047/

In [14]:
df_complete_saison[(df_complete_saison["H_TEAMNAMES"]=="Bayern Munich") & (df_complete_saison["A_TEAMNAMES"]=="Augsburg")]

,H_DATETIME,H_GAMEINFO,H_TEAMNAMES,H_GOALS,H_BALL_POSSESSION,H_GOAL_ATTEMPTS,H_SHOTS_ON_GOAL,H_SHOTS_OFF_GOAL,H_BLOCKED_SHOTS,H_FREE_KICKS,H_CORNER_KICKS,H_OFFSIDES,H_GOALKEEPER_SAVES,H_FOULS,H_YELLOW_CARDS,H_RED_CARDS,H_TOTAL_PASSES,H_COMPLETED_PASSES,H_TACKLES,H_ATTACKS,H_DANGEROUS_ATTACKS,A_DATETIME,A_GAMEINFO,A_TEAMNAMES,A_GOALS,A_BALL_POSSESSION,A_GOAL_ATTEMPTS,A_SHOTS_ON_GOAL,A_SHOTS_OFF_GOAL,A_BLOCKED_SHOTS,A_FREE_KICKS,A_CORNER_KICKS,A_OFFSIDES,A_GOALKEEPER_SAVES,A_FOULS,A_YELLOW_CARDS,A_RED_CARDS,A_TOTAL_PASSES,A_COMPLETED_PASSES,A_TACKLES,A_ATTACKS,A_DANGEROUS_ATTACKS,H_CAT,GAMEURL,H_INTERCEPTIONS,A_EXPECTED_GOALS_XG,H_THROWINS,H_EXPECTED_GOALS_XG,A_THROWINS,A_INTERCEPTIONS,H_CLEARANCES_TOTAL,H_SHOTS_INSIDE_THE_BOX,A_PASSES_IN_THE_FINAL_THIRD,A_CLEARANCES_TOTAL,H_CROSSES,H_PASSES_IN_THE_FINAL_THIRD,A_CROSSES,A_TOUCHES_IN_THE_OPPOSITION_BOX,H_SHOTS_OUTSIDE_THE_BOX,H_TOUCHES_IN_THE_OPPOSITION_BOX,H_BIG_CHANCES,A_BIG_CHANCES,H_HIT_THE_WOODWORK,A_SHOTS_OUTSIDE_THE_BOX,A_SHOTS_INSIDE_THE_BOX,A_HIT_THE_WOODWORK,H_HEADED_GOALS,A_HEADED_GOALS
99,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Bayern Munich,0,79,13,5,5,3.0,4,2,0,1,6,NaN,NaN,483,444,70.0,NaN,NaN,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Augsburg,0,21,2,1,1,0.0,6,0,0,5,4,NaN,NaN,NaN,NaN,67.0,NaN,NaN,cat89,https://www.flashscore.com/match/EuQnZBVQ/#/ma...,3,0.08,5,0.71,6,9,3,8,53.0,12,18.0,85.0,100.0,3,5,26,0,0,0,0,2,0,None,None
100,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Bayern Munich,0,81,20,9,7,4.0,5,6,1,0,3,0.0,0.0,400,368,33.0,NaN,NaN,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Augsburg,0,19,0,0,0,0.0,3,0,0,6,5,2.0,1.0,NaN,NaN,43.0,NaN,NaN,cat90,https://www.flashscore.com/match/EuQnZBVQ/#/ma...,1,0.02,9,3.06,2,4,0,15,38.0,20,20.0,85.0,0.0,0,5,27,4,0,1,0,0,0,1,0


In [10]:
asdasdas

NameError: name 'asdasdas' is not defined

In [10]:
def transform_column_name(name):
    # Spezialfälle direkt behandeln
    if name == 'GAMEURL':
        return 'gameurl'
    if name == 'H_CAT':
        return 'H_cat'
    
    # Prüfen auf Präfix (z.B. 'H_' oder 'A_')
    if '_' in name:
        prefix, *rest = name.split('_')
        # Spezielle Behandlung, wenn nur ein Teil nach dem Präfix kommt
        if len(rest) == 1:
            transformed_rest = rest[0].lower()
        else:
            # Join die restlichen Teile mit Leerzeichen und Titelkasierung
            transformed_rest = ' '.join(rest).replace('Xg', 'Xg').title()
            # Optional: Behandle spezifische Abkürzungen oder Fachbegriffe hier
        return f"{prefix}_{transformed_rest}"
    else:
        # Für Spalten ohne Unterstrich, einfach in Kleinbuchstaben umwandeln
        return name.lower()


In [11]:

df_complete_saison = df_complete_saison.replace(teamnamedict)
dfallteamnamesl = df_complete_saison.H_TEAMNAMES.unique()

# Schritt 3: Zuweisen von Punkten basierend auf dem Halbzeit-Ergebnis
def assign_points(row, team_type):
    if team_type == 'H': # and row['BP-H'] > 59
        if row['Halbzeit_Ergebnis_H'] == 'Gewinn':
            return 1
        elif row['Halbzeit_Ergebnis_H'] == 'Verlust':
            return -1
        else:
            return 0
    elif team_type == 'A': #and row['BP-A'] > 59
        if row['Halbzeit_Ergebnis_A'] == 'Gewinn':
            return 1
        elif row['Halbzeit_Ergebnis_A'] == 'Verlust':
            return -1
        else:
            return 0
    return 0

def post_process_table(df_filtered):
    df_filtered['Punkte_H'] = df_filtered.apply(lambda row: assign_points(row, 'H'), axis=1)
    df_filtered['Punkte_A'] = df_filtered.apply(lambda row: assign_points(row, 'A'), axis=1)

    # Schritt 4: Aggregieren der Punkte für jedes Team
    home_points = df_filtered.groupby('Home')['Punkte_H'].sum()
    away_points = df_filtered.groupby('Opponent')['Punkte_A'].sum()

    # Zusammenführen der Punkte für Heim- und Auswärtsteams
    total_points = home_points.add(away_points, fill_value=0).reset_index()
    total_points.columns = ['Team', 'Gesamtpunkte']

    # Zählen der Halbzeiten für jedes Team
    home_counts = df_filtered['Home'].value_counts()
    away_counts = df_filtered['Opponent'].value_counts()
    total_counts = home_counts.add(away_counts, fill_value=0)

    # Summieren der xG-Werte für und gegen jedes Team
    home_xg_for = df_filtered.groupby('Home')['xg_halftime'].sum()
    away_xg_for = df_filtered.groupby('Opponent')['Axg_halftime'].sum()
    home_xg_against = df_filtered.groupby('Home')['Axg_halftime'].sum()
    away_xg_against = df_filtered.groupby('Opponent')['xg_halftime'].sum()

    total_xg_for = home_xg_for.add(away_xg_for, fill_value=0)
    total_xg_against = home_xg_against.add(away_xg_against, fill_value=0)

    # Zusammenführen der Punkte und Halbzeitenanzahl in einem DataFrame
    total_points['Halbzeitenanzahl'] = total_points['Team'].map(total_counts)
    total_points['xG Für'] = total_points['Team'].map(total_xg_for)
    total_points['xG Gegen'] = total_points['Team'].map(total_xg_against)

    total_points['xG diff'] = total_points['xG Für'] - total_points['xG Gegen'] 

    total_points = total_points.sort_values(by=['Gesamtpunkte'], ascending=False)

    return total_points

def get_table_ballpositionstyle(dfxg_df_merged_cleaned):
    # Schritt 2: Filtern der Spiele mit Ballbesitz > 59%
    df_filtered = dfxg_df_merged_cleaned[(dfxg_df_merged_cleaned['BP-H'] > 56) ]
    # st.write("bp:")
    # st.write(df_filtered)
    # calculate the metrics for this selection of halftimes-df
    table_ballpositionstyle = post_process_table(df_filtered)
    return table_ballpositionstyle

def get_table_counterstyle(dfxg_df_merged_cleaned):
    df_filtered = dfxg_df_merged_cleaned[(dfxg_df_merged_cleaned['BP-H'] < 44) ]
    # st.write("counter:")
    # st.write(df_filtered)
    # calculate the metrics for this selection of halftimes-df
    table_counterstyle = post_process_table(df_filtered)
    return table_counterstyle

def get_table_even(dfxg_df_merged_cleaned):
    df_filtered = dfxg_df_merged_cleaned[(dfxg_df_merged_cleaned['BP-H'] <= 56) & (dfxg_df_merged_cleaned['BP-H'] >= 44)]
    # st.write("counter:")
    # st.write(df_filtered)
    # calculate the metrics for this selection of halftimes-df
    table_evenstyle = post_process_table(df_filtered)
    return table_evenstyle

# Schritt 1: Berechnen des Halbzeit-Ergebnisses für das Heimteam
def calc_halftime_result_h(row):
    goal_diff = row['G-H'] - row['G-A']
    if goal_diff > 0:
        return 'Gewinn'
    elif goal_diff < 0:
        return 'Verlust'
    else:
        return 'Unentschieden'
    
def calc_halftime_result_a(row):
    goal_diff = row['G-A'] - row['G-H']
    if goal_diff > 0:
        return 'Gewinn'
    elif goal_diff < 0:
        return 'Verlust'
    else:
        return 'Unentschieden'
            
def page_league_table():
    
    # TODO: iteriere über alle teams und packe alle ergebnisse in eine seperate liste und die kommt
    # dann in eine neue seperate ergebnis df!
    result_table_ballpositionstyle_list = []
    result_table_counterstyle_list = []
    result_table_evenstyle_list = []
    error_list = []

    for team in df_complete_saison["H_TEAMNAMES"].unique():
        try:
            # teamname corrected that it fits to htdatan teamnames
            # team = df_complete_saison["H_TEAMNAMES"].unique()[0]
            dfxg = load_xg_season_stats_sql(saison)

            # convert xg teamnames to correct ones that are used in htdatan
            dfxg = process_team_names_of_df(dfxg)

            # rename columns for
            dfxg_rename = dfxg.rename(
                columns={'TEAMS': 'H_TEAMNAMES', 'A_TEAMS': 'A_Teamnames'})

            dfxg_df_merged = pd.merge(
                df_complete_saison, dfxg_rename, on=["H_TEAMNAMES", "A_Teamnames"])
            df = dfxg_df_merged.drop_duplicates()

            # the following is only so the functions have the expecting columns
            df["homexg_complete_game"] = ""
            df["awayxg_complete_game"] = ""
            df["last_game_minute"] = -1
            df["start_min_game"] = -1

            dfxg_df_merged_cleaned = df_cleaning_converting(df)

            df4Home, df4OpponentReversed = df_specific_team(
                dfxg_df_merged_cleaned, team)
            dfxg_df_merged_cleaned = create_df4Complete(df4Home, df4OpponentReversed)

            # Sortieren des DataFrames nach Index (falls erforderlich)
            dfxg_df_merged_cleaned = dfxg_df_merged_cleaned.sort_index()

            # Berechnen der xG-Werte für die zweite Halbzeit und Aktualisieren der 'xg_halftime'-Spalte
            for i in range(1, len(dfxg_df_merged_cleaned), 2):  # Start bei 1 und springe jede zweite Zeile
                if dfxg_df_merged_cleaned.iloc[i]['Home'] == dfxg_df_merged_cleaned.iloc[i-1]['Home'] and dfxg_df_merged_cleaned.iloc[i]['Opponent'] == dfxg_df_merged_cleaned.iloc[i-1]['Opponent']:
                    dfxg_df_merged_cleaned.at[i, 'xg_halftime'] = dfxg_df_merged_cleaned.iloc[i]['xG'] - dfxg_df_merged_cleaned.iloc[i-1]['xg_halftime']
                    dfxg_df_merged_cleaned.at[i, 'Axg_halftime'] = dfxg_df_merged_cleaned.iloc[i]['A_xG'] - dfxg_df_merged_cleaned.iloc[i-1]['Axg_halftime']
            
            dfxg_df_merged_cleaned['Halbzeit_Ergebnis_H'] = dfxg_df_merged_cleaned.apply(calc_halftime_result_h, axis=1)
            dfxg_df_merged_cleaned['Halbzeit_Ergebnis_A'] = dfxg_df_merged_cleaned.apply(calc_halftime_result_a, axis=1)

            # now it should not be called Home Away but Team and Opposition!

            # st.write(dfxg_df_merged_cleaned)

            table_ballpositionstyle = get_table_ballpositionstyle(dfxg_df_merged_cleaned)
            table_counterstyle = get_table_counterstyle(dfxg_df_merged_cleaned)
            table_evenstyle = get_table_even(dfxg_df_merged_cleaned)

            # st.dataframe(table_ballpositionstyle[table_ballpositionstyle["Team"]==team])
            # st.dataframe(table_counterstyle[table_counterstyle["Team"]==team])
            # st.dataframe(table_evenstyle[table_evenstyle["Team"]==team])

            result_table_ballpositionstyle_list.append(table_ballpositionstyle[table_ballpositionstyle["Team"]==team])
            result_table_counterstyle_list.append(table_counterstyle[table_counterstyle["Team"]==team])
            result_table_evenstyle_list.append(table_evenstyle[table_evenstyle["Team"]==team])

        except:
            print(f"error for team {team}")
            error_list.append(team)


    # Concatenate all DataFrame rows into a final DataFrame
    result_table_ballpositionstyle = pd.concat(result_table_ballpositionstyle_list, ignore_index=True)
    result_table_counterstyle = pd.concat(result_table_counterstyle_list, ignore_index=True)
    result_table_evenstyle = pd.concat(result_table_evenstyle_list, ignore_index=True)
    
    result_table_ballpositionstyle = result_table_ballpositionstyle.sort_values(by=['Gesamtpunkte'], ascending=False)
    result_table_counterstyle = result_table_counterstyle.sort_values(by=['Gesamtpunkte'], ascending=False)
    result_table_evenstyle = result_table_evenstyle.sort_values(by=['Gesamtpunkte'], ascending=False)

    # Titel der Seite
    st.title("League table per play style")

    # Erstellen von drei Spalten
    col1, col2, col3 = st.columns(3)

    # Anzeigen des zweiten DataFrames in der zweiten Spalte
    with col1:
        st.header("Counter Style")  # Titel für das zweite DataFrame
        st.dataframe(result_table_counterstyle)

    # Anzeigen des dritten DataFrames in der dritten Spalte
    with col2:
        st.header("Even Style")  # Titel für das dritte DataFrame
        st.dataframe(result_table_evenstyle)

    # Anzeigen des ersten DataFrames in der ersten Spalte
    with col3:
        st.header("Ball Position Style")  # Titel für das erste DataFrame
        st.dataframe(result_table_ballpositionstyle)

    st.write(error_list)

# Define Functions:
def process_team_names_of_df(x_df):
    x_df = x_df.replace(teamnamedict)
    return x_df

#######################################################
###  calculate table with two halftimes to one game ###
#######################################################

def df_cleaning_converting(df):
    df = df[['H_TEAMNAMES', 'A_TEAMNAMES', 'H_GOALS', 'A_GOALS', 'H_BALL_POSSESSION', 'A_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'A_GOAL_ATTEMPTS',
            'H_SHOTS_ON_GOAL', 'A_SHOTS_ON_GOAL', 'H_SHOTS_OFF_GOAL', 'A_SHOTS_OFF_GOAL', 'H_FREE_KICKS', "H_RED_CARDS", "A_RED_CARDS",
            'A_FREE_KICKS', 'H_CORNER_KICKS', 'A_CORNER_KICKS', 'H_OFFSIDES', 'A_OFFSIDES', 'H_GOALKEEPER_SAVES', 'A_GOALKEEPER_SAVES',
            'H_FOULS', 'A_FOULS', 'A_GAMEINFO', 'A_DATETIME', 'XG', 'H_GOALS_COMPLETE_GAME', 'XPTS', 'A_XG', 'A_GOALS_COMPLETE_GAME', 'A_XPTS', 'TIMING_CHART_XG', "HOMEXG_COMPLETE_GAME", "AWAYXG_COMPLETE_GAME"]]
    df = df.drop_duplicates(subset=['H_TEAMNAMES', 'A_TEAMNAMES', 'H_GOALS', 'A_GOALS', 'H_BALL_POSSESSION', 'A_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'A_GOAL_ATTEMPTS',
            'H_SHOTS_ON_GOAL', 'A_SHOTS_ON_GOAL', 'H_SHOTS_OFF_GOAL', 'A_SHOTS_OFF_GOAL', 'H_FREE_KICKS', "H_RED_CARDS", "A_RED_CARDS",
            'A_FREE_KICKS', 'H_CORNER_KICKS', 'A_CORNER_KICKS', 'H_OFFSIDES', 'A_OFFSIDES', 'H_GOALKEEPER_SAVES', 'A_GOALKEEPER_SAVES',
            'H_FOULS', 'A_FOULS', 'A_GAMEINFO', 'A_DATETIME', 'XG', 'H_GOALS_COMPLETE_GAME', 'XPTS', 'A_XG', 'A_GOALS_COMPLETE_GAME', 'A_XPTS'], keep='first')
    df = df.reset_index(drop=True)
    df["R"] = 'X'

    df['TIMING_CHART_XG'] = df['TIMING_CHART_XG'].astype('str') 

    def extract_xg_values(df):
        """ Extracts the xG values from the TIMING_CHART_XG columnn.
            This contains a string formatted as a list of xG values for each minute.
        """
        for index, _ in df.iterrows():
            try:
                timing_chart = df["TIMING_CHART_XG"].loc[index]
                
                # Away team XG at halftime
                away_xg = None
                for minute in ['46', '45', '47']:
                    try:
                        if minute == '46':
                            away_xg = timing_chart.split("46' Total xG:")[0].split("Total xG: ")[-1].split("\n")[0].replace(";", "")
                        else:
                            away_xg = timing_chart.split(f"{int(minute)}' Total xG:")[0].split(f"{minute}' Total xG: ")[-1].split("\n")[0].replace(";", "")
                        break
                    except Exception:
                        if minute == '46':
                            print(f"Could not find 46' Total xG for away team at index {index}, trying 45' or 47'")
                        continue
                
                if away_xg is None:
                    print(f"Failed to extract away XG for all minute variations at index {index}")
                    continue
                
                # Home team XG at halftime
                home_xg = None
                for minute in ['45', '44', '46']:
                    try:
                        if minute == '45':
                            home_xg = timing_chart.split("45' Total xG: ")[1].split(";45' Total xG:")[0].split("\n")[0].replace(";", "")
                        else:
                            home_xg = timing_chart.split(f"{minute}' Total xG: ")[1].split(f";{minute}' Total xG:")[0].split("\n")[0].replace(";", "")
                        break
                    except Exception:
                        if minute == '45':
                            print(f"Could not find 45' Total xG for home team at index {index}, trying 44' or 46'")
                        continue
                
                if home_xg is None:
                    print(f"Failed to extract home XG for all minute variations at index {index}")
                    continue
                    
                # Assign values to dataframe
                df.loc[index, 'xg_halftime'] = home_xg
                df.loc[index, 'Axg_halftime'] = away_xg
                
            except Exception as e:
                print(f"Error processing index {index}: {str(e)}")
                continue
        
        return df

    df = extract_xg_values(df)


    df.xg_halftime = df.xg_halftime.astype(float).fillna(0.0)
    df.Axg_halftime = df.Axg_halftime.astype(float).fillna(0.0) 

    for i in range(0, len(df)):
        try:

            if df["H_Goals"][i] > df["A_Goals"][i]:
                df["R"][i] = 'H'
            if df["H_Goals"][i] < df["A_Goals"][i]:
                df["R"][i] = 'A'
            else:
                df["R"][i] = 'D'
        except:
            print("error?")
    print("!!!", df.columns)
    df.columns = ['Home', 'Opponent', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',"H_Red Cards", "A_Red Cards",
                'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A',
                'F-H', 'F-A', 'Round', 'Date', 'xG', 'H_GOALS_COMPLETE_GAME', 'xPTS', 'A_xG', 'A_GOALS_COMPLETE_GAME', 'A_xPTS', "timing_chart_xg", "homexg_complete_game", "awayxg_complete_game", 'R',  'xg_halftime', 'Axg_halftime']

    df = df[['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
             'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',"H_Red Cards", "A_Red Cards",
             'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
             'F-A', 'Round', 'Date', 'xG', 'H_GOALS_COMPLETE_GAME', 'xPTS', 'A_xG', 'A_GOALS_COMPLETE_GAME', 'A_xPTS', 'xg_halftime', 'Axg_halftime', "homexg_complete_game", "awayxg_complete_game"]]

    df["IsHome"] = 0

    df = df[['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
             'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',"H_Red Cards", "A_Red Cards",
             'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
             'F-A', 'Round', 'Date', 'IsHome', 'xG', 'H_GOALS_COMPLETE_GAME', 'xPTS', 'A_xG', 'A_GOALS_COMPLETE_GAME', 'A_xPTS','xg_halftime', 'Axg_halftime', "homexg_complete_game", "awayxg_complete_game", ]]
    return df


@st.cache(suppress_st_warning=True)
def df_specific_team(df, team):
    df4Home = df.loc[((df['Home'] == team))]

    df4Home["IsHome"] = 1

    # recalcualte the winner because of the columns switching to bring the selected team in the first column
    df4Home["1x2"] = 0

    df4Home["1x2"] = df4Home.apply(
        lambda row: calculate_1x2_home(row), axis=1, result_type='reduce')

    # Berechnung Opponentgames
    df4Opponent = df.loc[((df['Opponent'] == team))]

    # recalcualte the winner because of the columns switching to bring the selected team in the first column
    df4Opponent["1x2"] = 0

    df4Opponent["1x2"] = df4Opponent.apply(
        lambda row: calculate_1x2_Opponent(row), axis=1, result_type='reduce')

    # change the halftime-xg with halftime-Axg:
    # switched the two columns
    # Change the columns for the Opponentmatches of the specific team
    print("df4Opponent.columns before reassignment oppo", df4Opponent.columns)

    OpponentTeamReversedColumns = ['Opponent', 'Home',  '1x2', 'R',  'G-A', 'G-H', 'BP-A', 'BP-H', 'GA-A', 'GA-H',  
                                    'SoG-A', 'SoG-H', 'SoffG-A', 'SoffG-H',  'FK-A',  "A_Red Cards", "H_Red Cards",
                                    'FK-H','C-A', 'C-H',  'Off-A', 'Off-H', 'GoKeSa-A', 'GoKeSa-H', 'F-A', 
                                    'F-H', 'Round', 'Date', 'IsHome','A_xG', "A_xPTS", "A_GOALS_COMPLETE_GAME", 'xG', "xPTS", "H_GOALS_COMPLETE_GAME", 'Axg_halftime', 'xg_halftime',"awayxg_complete_game", "homexg_complete_game",   ]  # , 'IsHome'
    # Change the columns for the Opponentmatches of the specific team

    df4OpponentReversed = df4Opponent.reindex(
        columns=OpponentTeamReversedColumns)

    df4OpponentReversed.columns = ['Home', 'Opponent', '1x2', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                                   'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H', "H_Red Cards", "A_Red Cards",
                                   'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
                                   'F-A', 'Round', 'Date', 'IsHome', 'xG', "xPTS", "H_GOALS_COMPLETE_GAME", 'A_xG', "A_xPTS", "A_GOALS_COMPLETE_GAME", 'xg_halftime', 'Axg_halftime', "homexg_complete_game", "awayxg_complete_game", ]

    print("df4OpponentReversed.columns after reassignment oppo", df4OpponentReversed.columns)

    return df4Home, df4OpponentReversed

@st.cache(suppress_st_warning=True)
def create_df4Complete(df4Home, df4OpponentReversed):
    # Alle Spiele werden als Heimspiel angezeigt, sind aber auch Auswärtsspiele dabei!
    df4Complete = pd.concat([df4Home, df4OpponentReversed], sort=False)

    df4Complete["G-H"] = df4Complete["G-H"].astype('float64')
    df4Complete["G-A"] = df4Complete["G-A"].astype('float64')
    df4Complete["BP-H"] = df4Complete["BP-H"].astype('float64')
    df4Complete["BP-A"] = df4Complete["BP-A"].astype('float64')

    # GoalDifference
    df4Complete["GoalDiff"] = df4Complete["G-H"] - df4Complete["G-A"]
    df4Complete = df4Complete.sort_values("Date",  ascending=False)

    # calculate column with 3 Ballposition types
    df4Complete["BPTypes"] = '0'

    df4Complete["BPTypes"] = df4Complete.apply(
        lambda row: calculate_1x2_BPTypes(row), axis=1, result_type='reduce')

    df4Complete['Date'] = pd.to_datetime(
        df4Complete['Date'], format="%d.%m.%Y %H:%M")

    # convert datetime to timestamp for scatter visualization
    df4Complete['timestamp'] = df4Complete.Date.astype('int64')//10**9

    df4Complete = df4Complete.sort_values("Date", ascending=False)

    # Create data for scatter graph
    df4Complete["SoG-H"] = df4Complete["SoG-H"].astype(int)
    df4Complete["SoG-A"] = df4Complete["SoG-A"].astype(int)

    return df4Complete


@st.cache
def calculate_1x2_home(row):
    if row['G-H'] > row['G-A']:
        return 'W'
    elif row['G-A'] > row['G-H']:
        return 'L'
    else:
        return 'D'

@st.cache
def calculate_1x2_Opponent(row):
    if row['G-A'] > row['G-H']:
        return 'W'
    elif row['G-H'] > row['G-A']:
        return 'L'
    else:
        return 'D'

@st.cache
def calculate_1x2_BPTypes(row):
    if row['BP-H'] > 55:
        return '>55'
    elif row['BP-H'] < 45:
        return '<45'
    else:
        return '45-55'

@st.cache
def calc_stats(df4Complete):
    BP_WPerc = df4Complete[['BPTypes', '1x2']
                           ].loc[df4Complete['BPTypes'] == '>55']
    BP_WAbs = BP_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    BP_NWAbs = BP_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(BP_WAbs) + len(BP_NWAbs) > 0:
        BP_WPercText = len(BP_WAbs) / (len(BP_WAbs) + len(BP_NWAbs)) * 100
        BP_WPercText = round(BP_WPercText)
    else:
        BP_WPercText = 0

    # calculate winning % for 0.45 - 0.55
    N_WPerc = df4Complete[['BPTypes', '1x2']
                          ].loc[df4Complete['BPTypes'] == '45-55']
    N_WAbs = N_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    N_NWAbs = N_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(N_WAbs) + len(N_NWAbs) > 0:
        N_WPercText = len(N_WAbs) / (len(N_WAbs) + len(N_NWAbs)) * 100
        N_WPercText = round(N_WPercText)
    else:
        N_WPercText = 5

    # calculate winning % for < 0.45
    C_WPerc = df4Complete[['BPTypes', '1x2']
                          ].loc[df4Complete['BPTypes'] == '<45']
    C_WAbs = C_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    C_NWAbs = C_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(C_WAbs) + len(C_NWAbs) > 0:
        C_WPercText = len(C_WAbs) / (len(C_WAbs) + len(C_NWAbs)) * 100
        C_WPercText = round(C_WPercText)
    else:
        C_WPercText = 0

    return C_WPercText, N_WPercText, BP_WPercText


def convert_htdatan_to_xg_table_name(htdatan_table_name):
    # Mapping from league codes to full league names
    league_mapping = {
        "B": "BUNDESLIGA",
        "PL": "EPL",
        "LL": "LA_LIGA",
        "L1": "LIGUE_1",
        "SA": "SERIE_A"
    }

    # Split the input into league code and season
    league_code, season = htdatan_table_name.upper().split("_")
    # Convert season to the correct year
    start_year = "20" + season[:2]
    full_season = start_year
    # Map league code to full league name
    full_league_name = league_mapping.get(league_code, league_code)
    # Format and return the output
    return f"XG_{full_league_name}{full_season}"

# get name of the selected team in dropdown
def load_xg_gamestats_oracle(saison, team):
    xg_table_name = convert_htdatan_to_xg_table_name(saison)
    df_complete_saison = db.show_table(xg_table_name)

    df_complete_saison = process_team_names_of_df(df_complete_saison)

    # execute the query and assign it to a pandas dataframe
    dfxg = df_complete_saison[(df_complete_saison.TEAMS == team) | (
        df_complete_saison.A_TEAMS == team)]
    return dfxg


# get name of the selected team in dropdown
def load_xg_gamestats_sql(saison, team):

    if saison.split("_")[0] == 'b':
        xgprefix = 'bundesliga'
    elif saison.split("_")[0] == 'l1':
        xgprefix = 'ligue_1'
    elif saison.split("_")[0] == 'll':
        xgprefix = 'la_liga'
    elif saison.split("_")[0] == 'pl':
        xgprefix = 'epl'
    elif saison.split("_")[0] == 'sa':
        xgprefix = 'serie_a'

    xgtablename = "{}20{}".format(xgprefix, saison.split("_")[1][:2])

    df_complete_saison = pd.read_csv(
        "data/xg/"+xgtablename+".csv", index_col=0, encoding='utf-8')

    df_complete_saison = process_team_names_of_df(df_complete_saison)

    # execute the query and assign it to a pandas dataframe
    dfxg = df_complete_saison[(df_complete_saison.TEAMS == team) | (
        df_complete_saison.A_TEAMS == team)]

    return dfxg

# get all teams for the selected season in dropdown
def load_xg_season_stats_sql(saison):

    if saison.split("_")[0] == 'b':
        xgprefix = 'bundesliga'
    elif saison.split("_")[0] == 'l1':
        xgprefix = 'ligue_1'
    elif saison.split("_")[0] == 'll':
        xgprefix = 'la_liga'
    elif saison.split("_")[0] == 'pl':
        xgprefix = 'epl'
    elif saison.split("_")[0] == 'sa':
        xgprefix = 'serie_a'

    xgtablename = "{}20{}".format(xgprefix, saison.split("_")[1][:2])

    df_complete_saison = pd.read_csv(
        "data/xg/"+xgtablename+".csv", index_col=0, encoding='utf-8')

    df_complete_saison = process_team_names_of_df(df_complete_saison)

    return df_complete_saison



2025-01-11 17:11:31.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.529 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2025-01-11 17:11:31.529 No runtime found, using MemoryCacheStorageManager
2025-01-11 17:11:31.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.534 Thread 'MainThread': missing ScriptRu

In [12]:
# take only the 0 part of the every list entry
teamsoptions = []
for x in range(0, len(dfallteamnamesl)):
    teamsoptions.append(dfallteamnamesl[x])

global xg_team
xg_team = st.sidebar.selectbox("Team", list(np.sort(teamsoptions)), 0)

# convert string to df to use process_team_names_of_df function
df_teamname = pd.DataFrame([xg_team])

# convert xg teamnames to correct ones that are used in htdatan
team_df = df_teamname.replace(teamnamedict)
# teamname corrected that it fits to htdatan teamnames
team = team_df.iloc[0][0]

df = df_complete_saison[(df_complete_saison.H_TEAMNAMES == team) | (
    df_complete_saison.A_TEAMNAMES == team)]

df = process_team_names_of_df(df)

2025-01-11 17:11:31.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 17:11:31.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [13]:
saison, team

('b_2425', 'Augsburg')

In [14]:
dfxg = load_xg_gamestats_oracle(saison, team)

In [15]:
dfxg[['TEAMS', 'GOALS', 'XG', 'SHOTS', 'SHOTS_ON_TARGET', 'DEEP', 'PPDA',
       'XPTS', 'A_TEAMS', 'A_GOALS', 'A_XG', 'A_SHOTS', 'A_SHOTS_ON_TARGET',
       'A_DEEP', 'A_PPDA', 'A_XPTS', 'GAMEURL', 'ID']]

,TEAMS,GOALS,XG,SHOTS,SHOTS_ON_TARGET,DEEP,PPDA,XPTS,A_TEAMS,A_GOALS,A_XG,A_SHOTS,A_SHOTS_ON_TARGET,A_DEEP,A_PPDA,A_XPTS,GAMEURL,ID
6,Augsburg,2,1.3,11,3,3,14.8,1.2,Werder Bremen,2,1.5,11,5,11,10.8,1.5,https://understat.com/match/27746,7
12,Augsburg,2,2.8,29,8,16,4.2,2.4,Mainz,3,1.3,5,3,1,23.8,0.5,https://understat.com/match/27769,13
13,FC Heidenheim,4,3.1,15,8,7,13.1,2.8,Augsburg,0,0.6,15,1,10,7.6,0.1,https://understat.com/match/27758,14
22,Augsburg,3,2.6,18,9,5,13.6,2.7,St. Pauli,1,0.5,11,3,9,9.7,0.2,https://understat.com/match/27767,23
25,Augsburg,2,1.8,15,5,8,8.8,2.1,B. Monchengladbach,1,0.8,11,4,5,4.8,0.7,https://understat.com/match/27790,26
40,RB Leipzig,4,2.8,17,7,13,14.5,2.5,Augsburg,0,1.1,7,1,4,15.9,0.3,https://understat.com/match/27783,41
56,Freiburg,3,1.1,15,6,3,16.6,1.5,Augsburg,1,0.9,9,3,4,18.0,1.2,https://understat.com/match/27800,57
74,Wolfsburg,1,1.1,18,6,14,8.4,1.9,Augsburg,1,0.5,3,1,1,18.7,0.8,https://understat.com/match/27820,75
77,Augsburg,2,0.6,12,4,7,16.1,1.1,Borussia Dortmund,1,0.8,11,4,5,5.6,1.5,https://understat.com/match/27809,78


In [16]:
asdasd

# hier für auswährtsspiele schon falsch
nur bei auswärtsspiele wird immer doppelt die erste hz von tore gegentore gesehen. also zweite hz ist immer gleich wie erste bei auswärtsspiele


NameError: name 'asdasd' is not defined

In [22]:
# show all the columns of the dataframe when i print
pd.set_option('display.max_columns', None)

df[['H_DATETIME', 'H_GAMEINFO', 'H_TEAMNAMES', 'H_GOALS',
       'H_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'H_SHOTS_ON_GOAL',
       'H_SHOTS_OFF_GOAL', 'H_BLOCKED_SHOTS', 'H_FREE_KICKS', 'H_CORNER_KICKS',
       'H_OFFSIDES', 'H_GOALKEEPER_SAVES', 'H_FOULS', 'H_YELLOW_CARDS',
       'H_RED_CARDS', 'H_TOTAL_PASSES', 'H_COMPLETED_PASSES', 'H_TACKLES',
       'H_ATTACKS', 'H_DANGEROUS_ATTACKS', 'A_DATETIME', 'A_GAMEINFO',
       'A_TEAMNAMES', 'A_GOALS', 'A_BALL_POSSESSION', 'A_GOAL_ATTEMPTS',
       'A_SHOTS_ON_GOAL', 'A_SHOTS_OFF_GOAL', 'A_BLOCKED_SHOTS',
       'A_FREE_KICKS', 'A_CORNER_KICKS', 'A_OFFSIDES', 'A_GOALKEEPER_SAVES',
       'A_FOULS', 'A_YELLOW_CARDS', 'A_RED_CARDS', 'A_TOTAL_PASSES',
       'A_COMPLETED_PASSES', 'A_TACKLES', 'A_ATTACKS', 'A_DANGEROUS_ATTACKS',
       'H_CAT', 'GAMEURL', 'H_INTERCEPTIONS', 'A_EXPECTED_GOALS_XG',
       'H_THROWINS', 'H_EXPECTED_GOALS_XG', 'A_THROWINS', 'A_INTERCEPTIONS',
       'H_CLEARANCES_TOTAL', 'H_SHOTS_INSIDE_THE_BOX',
       'A_PASSES_IN_THE_FINAL_THIRD', 'A_CLEARANCES_TOTAL', 'H_CROSSES',
       'H_PASSES_IN_THE_FINAL_THIRD', 'A_CROSSES',
       'H_SHOTS_OUTSIDE_THE_BOX',
       'H_BIG_CHANCES', 'A_BIG_CHANCES',
       'H_HIT_THE_WOODWORK', 'A_SHOTS_OUTSIDE_THE_BOX',
       'A_SHOTS_INSIDE_THE_BOX', 'A_HIT_THE_WOODWORK', 'H_HEADED_GOALS',
       'A_HEADED_GOALS']].tail(6)

,H_DATETIME,H_GAMEINFO,H_TEAMNAMES,H_GOALS,H_BALL_POSSESSION,H_GOAL_ATTEMPTS,H_SHOTS_ON_GOAL,H_SHOTS_OFF_GOAL,H_BLOCKED_SHOTS,H_FREE_KICKS,H_CORNER_KICKS,H_OFFSIDES,H_GOALKEEPER_SAVES,H_FOULS,H_YELLOW_CARDS,H_RED_CARDS,H_TOTAL_PASSES,H_COMPLETED_PASSES,H_TACKLES,H_ATTACKS,H_DANGEROUS_ATTACKS,A_DATETIME,A_GAMEINFO,A_TEAMNAMES,A_GOALS,A_BALL_POSSESSION,A_GOAL_ATTEMPTS,A_SHOTS_ON_GOAL,A_SHOTS_OFF_GOAL,A_BLOCKED_SHOTS,A_FREE_KICKS,A_CORNER_KICKS,A_OFFSIDES,A_GOALKEEPER_SAVES,A_FOULS,A_YELLOW_CARDS,A_RED_CARDS,A_TOTAL_PASSES,A_COMPLETED_PASSES,A_TACKLES,A_ATTACKS,A_DANGEROUS_ATTACKS,H_CAT,GAMEURL,H_INTERCEPTIONS,A_EXPECTED_GOALS_XG,H_THROWINS,H_EXPECTED_GOALS_XG,A_THROWINS,A_INTERCEPTIONS,H_CLEARANCES_TOTAL,H_SHOTS_INSIDE_THE_BOX,A_PASSES_IN_THE_FINAL_THIRD,A_CLEARANCES_TOTAL,H_CROSSES,H_PASSES_IN_THE_FINAL_THIRD,A_CROSSES,H_SHOTS_OUTSIDE_THE_BOX,H_BIG_CHANCES,A_BIG_CHANCES,H_HIT_THE_WOODWORK,A_SHOTS_OUTSIDE_THE_BOX,A_SHOTS_INSIDE_THE_BOX,A_HIT_THE_WOODWORK,H_HEADED_GOALS,A_HEADED_GOALS
222,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Bayern Munich,0,79,13,5,5,3.0,4,2,0,1,6,NaN,NaN,483,444,70.0,NaN,NaN,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Augsburg,0,21,2,1,1,0.0,6,0,0,5,4,NaN,NaN,NaN,NaN,67.0,NaN,NaN,cat89,https://www.flashscore.com/match/EuQnZBVQ/#/ma...,3,0.08,5,0.71,6,9,3,8,53.0,12,18.0,85.0,100.0,5,0,0,0,0,2,0,None,None
223,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Bayern Munich,0,81,20,9,7,4.0,5,6,1,0,3,0.0,0.0,400,368,33.0,NaN,NaN,22.11.2024 19:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 11,Augsburg,0,19,0,0,0,0.0,3,0,0,6,5,2.0,1.0,NaN,NaN,43.0,NaN,NaN,cat90,https://www.flashscore.com/match/EuQnZBVQ/#/ma...,1,0.02,9,3.06,2,4,0,15,38.0,20,20.0,85.0,0.0,5,4,0,1,0,0,0,1,0
228,10.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 10,Augsburg,0,53,11,2,4,5.0,4,5,2,0,5,0.0,NaN,270,231,NaN,NaN,NaN,10.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 10,Hoffenheim,0,47,4,0,1,3.0,5,5,0,2,4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,cat95,https://www.flashscore.com/match/UJQITUyi/#/ma...,7,0.11,1,0.44,8,6,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None
229,10.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 10,Augsburg,0,35,5,2,1,2.0,3,5,0,2,11,2.0,NaN,172,129,NaN,NaN,NaN,10.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 10,Hoffenheim,0,65,7,2,3,2.0,11,1,3,2,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,cat96,https://www.flashscore.com/match/UJQITUyi/#/ma...,10,0.32,9,0.44,17,6,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None
249,26.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 8,Augsburg,1,18,2,1,1,0.0,2,2,1,1,9,1.0,NaN,94,64,NaN,NaN,NaN,26.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 8,Borussia Dortmund,1,82,4,2,1,1.0,9,2,0,0,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,cat135,https://www.flashscore.com/match/x0haJmIE/#/ma...,7,0.60,6,0.07,9,3,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None
250,26.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 8,Augsburg,1,28,10,3,6,1.0,8,1,0,2,4,2.0,0.0,125,86,NaN,NaN,NaN,26.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 8,Borussia Dortmund,1,72,7,2,3,2.0,4,3,1,2,8,3.0,1.0,NaN,NaN,NaN,NaN,NaN,cat136,https://www.flashscore.com/match/x0haJmIE/#/ma...,7,0.32,9,0.48,4,1,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None


In [ ]:
# rename columns for
dfxg_rename = dfxg.rename(
    columns={'TEAMS': 'H_TEAMNAMES', 'A_TEAMS': 'A_TEAMNAMES'})
# del dfxg

dfxg_df_merged = pd.merge(
    df, dfxg_rename, on=["H_TEAMNAMES", "A_TEAMNAMES"])
dfxg_df_merged = dfxg_df_merged.drop_duplicates()

In [21]:
# show all the columns of the dataframe when i print
pd.set_option('display.max_columns', None)


dfxg_df_merged[['H_DATETIME', 'H_GAMEINFO', 'H_TEAMNAMES', 'H_GOALS',
       'H_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'H_SHOTS_ON_GOAL',
       'H_SHOTS_OFF_GOAL', 'H_BLOCKED_SHOTS', 'H_FREE_KICKS', 'H_CORNER_KICKS',
       'H_OFFSIDES', 'H_GOALKEEPER_SAVES', 'H_FOULS', 'H_YELLOW_CARDS',
       'H_RED_CARDS', 'H_TOTAL_PASSES', 'H_COMPLETED_PASSES', 'H_TACKLES',
       'H_ATTACKS', 'H_DANGEROUS_ATTACKS', 'A_DATETIME', 'A_GAMEINFO',
       'A_TEAMNAMES', 'A_GOALS_x', 'A_BALL_POSSESSION', 'A_GOAL_ATTEMPTS',
       'A_SHOTS_ON_GOAL', 'A_SHOTS_OFF_GOAL', 'A_BLOCKED_SHOTS',
       'A_FREE_KICKS', 'A_CORNER_KICKS', 'A_OFFSIDES', 'A_GOALKEEPER_SAVES',
       'A_FOULS', 'A_YELLOW_CARDS', 'A_RED_CARDS', 'A_TOTAL_PASSES',
       'A_COMPLETED_PASSES', 'A_TACKLES', 'A_ATTACKS', 'A_DANGEROUS_ATTACKS',
       'H_CAT', 'GAMEURL_x', 'H_INTERCEPTIONS', 'A_EXPECTED_GOALS_XG',
       'H_THROWINS', 'H_EXPECTED_GOALS_XG', 'A_THROWINS', 'A_INTERCEPTIONS',
       'H_CLEARANCES_TOTAL', 'H_SHOTS_INSIDE_THE_BOX',
       'A_PASSES_IN_THE_FINAL_THIRD', 'A_CLEARANCES_TOTAL', 'H_CROSSES',
       'H_PASSES_IN_THE_FINAL_THIRD', 'A_CROSSES', 'H_BIG_CHANCES', 'A_BIG_CHANCES',
       'H_HIT_THE_WOODWORK', 'A_SHOTS_OUTSIDE_THE_BOX',
       'A_SHOTS_INSIDE_THE_BOX', 'A_HIT_THE_WOODWORK', 'H_HEADED_GOALS',
       'A_HEADED_GOALS', 'GOALS', 'XG', 'SHOTS', 'SHOTS_ON_TARGET', 'DEEP',
       'PPDA', 'XPTS', 'A_GOALS_y', 'A_XG', 'A_SHOTS', 'A_SHOTS_ON_TARGET',
       'A_DEEP', 'A_PPDA', 'A_XPTS', 'GAMEURL_y', 'ID']].tail(10)

NameError: name 'dfxg_df_merged' is not defined

In [ ]:


df = dfxg_df_merged

df["homexg_complete_game"] = ""
df["awayxg_complete_game"] = ""
df["last_game_minute"] = -1
df["start_min_game"] = -1

        
for game_loc in df.index:

    homexg_complete_game = []
    awayxg_complete_game = []

    last_game_minute = df["TIMING_CHART_XG"].loc[game_loc].rsplit("'")[-2].rsplit(";")[1]
    start_min_game = int( re.sub("[^0-9]", "", df["TIMING_CHART_XG"].loc[game_loc][:2]) )

    # nehmen hier minute für minute und schauen nach dem xg wert für diese minute
    for x in range(start_min_game,int(last_game_minute)+1):
        try:
            homexgperminute = df["TIMING_CHART_XG"].loc[game_loc].split("{}' Total xG: ".format(x))[1].split(";")[0][:4]  # [:4] - only last 4 digits so no goalscorer infos
            awayxgperminute = df["TIMING_CHART_XG"].loc[game_loc].split("{}' Total xG: ".format(x), 2)[2].split(";")[0][:4]
        except:
            # falls die minute fehlt nehmen wir einfach den xg wert von der vorherigen minute!
            print("min {} is missing in xg".format(x))

        homexg_complete_game.append(homexgperminute)
        awayxg_complete_game.append(awayxgperminute)
        
    df["homexg_complete_game"].loc[game_loc] = homexg_complete_game
    df["awayxg_complete_game"].loc[game_loc] = awayxg_complete_game
    df["last_game_minute"].loc[game_loc] = last_game_minute
    df["start_min_game"].loc[game_loc] = start_min_game

for x in range(len(df)):
    df.homexg_complete_game.iloc[x][0:0] = [None] * df.start_min_game.iloc[x]
    df.awayxg_complete_game.iloc[x][0:0] = [None] * df.start_min_game.iloc[x]


df_homexg_complete_game = pd.DataFrame(df.homexg_complete_game.tolist(), index= df.index)
df_awayxg_complete_game = pd.DataFrame(df.awayxg_complete_game.tolist(), index= df.index)

df_homexg_complete_game = df_homexg_complete_game.apply(pd.to_numeric)
df_awayxg_complete_game = df_awayxg_complete_game.apply(pd.to_numeric)

df

min 19 is missing in xg
min 41 is missing in xg
min 63 is missing in xg
min 85 is missing in xg
min 19 is missing in xg
min 41 is missing in xg
min 63 is missing in xg
min 85 is missing in xg
min 19 is missing in xg
min 41 is missing in xg
min 63 is missing in xg
min 85 is missing in xg
min 19 is missing in xg
min 41 is missing in xg
min 63 is missing in xg
min 85 is missing in xg
min 74 is missing in xg
min 74 is missing in xg
min 26 is missing in xg
min 39 is missing in xg
min 53 is missing in xg
min 67 is missing in xg
min 81 is missing in xg
min 26 is missing in xg
min 39 is missing in xg
min 53 is missing in xg
min 67 is missing in xg
min 81 is missing in xg
min 17 is missing in xg
min 26 is missing in xg
min 35 is missing in xg
min 44 is missing in xg
min 53 is missing in xg
min 62 is missing in xg
min 71 is missing in xg
min 80 is missing in xg
min 89 is missing in xg
min 17 is missing in xg
min 26 is missing in xg
min 35 is missing in xg
min 44 is missing in xg
min 53 is missin

,H_DATETIME,H_GAMEINFO,H_TEAMNAMES,H_GOALS,H_BALL_POSSESSION,H_GOAL_ATTEMPTS,H_SHOTS_ON_GOAL,H_SHOTS_OFF_GOAL,H_BLOCKED_SHOTS,H_FREE_KICKS,...,A_DEEP,A_PPDA,A_XPTS,TIMING_CHART_XG,GAMEURL_y,ID,homexg_complete_game,awayxg_complete_game,last_game_minute,start_min_game
0,02.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,Wolfsburg,0,68,10,3,2,5.0,5,...,1,18.7,0.8,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27820,75,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.01, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...",90,0
1,02.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,Wolfsburg,0,72,8,3,3,2.0,9,...,1,18.7,0.8,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27820,75,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.01, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...",90,0
2,24.08.2024 15:30,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 1,Augsburg,0,49,4,0,4,NaN,3,...,11,10.8,1.5,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27746,7,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...",90,0
3,24.08.2024 15:30,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 1,Augsburg,2,41,6,3,3,NaN,11,...,11,10.8,1.5,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27746,7,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...",90,0
4,28.09.2024 15:30,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 5,RB Leipzig,2,58,10,4,6,NaN,9,...,4,15.9,0.3,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27783,41,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...",90,0
5,28.09.2024 15:30,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 5,RB Leipzig,2,56,4,3,1,NaN,7,...,4,15.9,0.3,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27783,41,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...",90,0
6,19.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 7,Freiburg,3,65,10,5,3,2.0,4,...,4,18.0,1.2,0' Total xG: 0.00;0' Total xG: 0.01;0' Total x...,https://understat.com/match/27800,57,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0...",93,0
7,19.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 7,Freiburg,3,49,5,1,1,3.0,3,...,4,18.0,1.2,0' Total xG: 0.00;0' Total xG: 0.01;0' Total x...,https://understat.com/match/27800,57,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0...",93,0
8,26.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 8,Augsburg,1,18,2,1,1,0.0,2,...,5,5.6,1.5,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27809,78,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.0...","[0.00, 0.00, 0.00, 0.40, 0.40, 0.40, 0.40, 0.4...",97,0
9,26.10.2024 13:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 8,Augsburg,1,28,10,3,6,1.0,8,...,5,5.6,1.5,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27809,78,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.0...","[0.00, 0.00, 0.00, 0.40, 0.40, 0.40, 0.40, 0.4...",97,0


In [17]:
# # show complete df cell
# pd.set_option('display.max_colwidth', None)
# # rename 'A_GOALS_x' --> 'A_GOALS'
df.rename(columns={'A_GOALS_x': 'A_GOALS'}, inplace=True)
df.rename(columns={'A_GOALS_y': 'A_GOALS_COMPLETE_GAME'}, inplace=True)
df.rename(columns={'GOALS': 'H_GOALS_COMPLETE_GAME'}, inplace=True)
df.rename(columns={'homexg_complete_game': 'HOMEXG_COMPLETE_GAME'}, inplace=True)
df.rename(columns={'awayxg_complete_game': 'AWAYXG_COMPLETE_GAME'}, inplace=True)



In [18]:
df.columns

Index(['H_DATETIME', 'H_GAMEINFO', 'H_TEAMNAMES', 'H_GOALS',
       'H_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'H_SHOTS_ON_GOAL',
       'H_SHOTS_OFF_GOAL', 'H_BLOCKED_SHOTS', 'H_FREE_KICKS', 'H_CORNER_KICKS',
       'H_OFFSIDES', 'H_GOALKEEPER_SAVES', 'H_FOULS', 'H_YELLOW_CARDS',
       'H_RED_CARDS', 'H_TOTAL_PASSES', 'H_COMPLETED_PASSES', 'H_TACKLES',
       'H_ATTACKS', 'H_DANGEROUS_ATTACKS', 'A_DATETIME', 'A_GAMEINFO',
       'A_TEAMNAMES', 'A_GOALS', 'A_BALL_POSSESSION', 'A_GOAL_ATTEMPTS',
       'A_SHOTS_ON_GOAL', 'A_SHOTS_OFF_GOAL', 'A_BLOCKED_SHOTS',
       'A_FREE_KICKS', 'A_CORNER_KICKS', 'A_OFFSIDES', 'A_GOALKEEPER_SAVES',
       'A_FOULS', 'A_YELLOW_CARDS', 'A_RED_CARDS', 'A_TOTAL_PASSES',
       'A_COMPLETED_PASSES', 'A_TACKLES', 'A_ATTACKS', 'A_DANGEROUS_ATTACKS',
       'H_CAT', 'GAMEURL_x', 'H_INTERCEPTIONS', 'A_EXPECTED_GOALS_XG',
       'H_THROWINS', 'H_EXPECTED_GOALS_XG', 'A_THROWINS', 'A_INTERCEPTIONS',
       'H_CLEARANCES_TOTAL', 'H_SHOTS_INSIDE_THE_BOX',
       'A_

In [ ]:
# pd show all columns
pd.set_option('display.max_columns', None)

df[['H_DATETIME', 'H_GAMEINFO', 'H_TEAMNAMES', 'H_GOALS',
       'H_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'H_SHOTS_ON_GOAL',
       'H_SHOTS_OFF_GOAL', 'H_BLOCKED_SHOTS', 'H_FREE_KICKS', 'H_CORNER_KICKS',
       'H_OFFSIDES', 'H_GOALKEEPER_SAVES', 'H_FOULS', 'H_YELLOW_CARDS',
       'H_RED_CARDS', 'H_TOTAL_PASSES', 'H_COMPLETED_PASSES', 'H_TACKLES',
       'H_ATTACKS', 'H_DANGEROUS_ATTACKS', 'A_DATETIME', 'A_GAMEINFO',
       'A_TEAMNAMES', 'A_GOALS', 'A_BALL_POSSESSION', 'A_GOAL_ATTEMPTS',
       'A_SHOTS_ON_GOAL', 'A_SHOTS_OFF_GOAL', 'A_BLOCKED_SHOTS',
       'A_FREE_KICKS', 'A_CORNER_KICKS', 'A_OFFSIDES', 'A_GOALKEEPER_SAVES',
       'A_FOULS', 'A_YELLOW_CARDS', 'A_RED_CARDS', 'A_TOTAL_PASSES',
       'A_COMPLETED_PASSES', 'A_TACKLES', 'A_ATTACKS', 'A_DANGEROUS_ATTACKS',
       'H_CAT', 'GAMEURL_x', 'H_INTERCEPTIONS', 'A_EXPECTED_GOALS_XG',
       'H_THROWINS', 'H_EXPECTED_GOALS_XG', 'A_THROWINS', 'A_INTERCEPTIONS',
       'H_CLEARANCES_TOTAL', 'H_SHOTS_INSIDE_THE_BOX',
       'A_PASSES_IN_THE_FINAL_THIRD', 'A_CLEARANCES_TOTAL', 'H_CROSSES',
       'H_PASSES_IN_THE_FINAL_THIRD', 'A_CROSSES',
       'A_TOUCHES_IN_THE_OPPOSITION_BOX', 'H_SHOTS_OUTSIDE_THE_BOX',
       'H_TOUCHES_IN_THE_OPPOSITION_BOX', 'H_BIG_CHANCES', 'A_BIG_CHANCES',
       'H_HIT_THE_WOODWORK', 'A_SHOTS_OUTSIDE_THE_BOX',
       'A_SHOTS_INSIDE_THE_BOX', 'A_HIT_THE_WOODWORK', 'H_HEADED_GOALS',
       'A_HEADED_GOALS', 'H_GOALS_COMPLETE_GAME', 'XG', 'SHOTS', 'SHOTS_ON_TARGET', 'DEEP',
       'PPDA', 'XPTS', 'A_GOALS_COMPLETE_GAME', 'A_XG', 'A_SHOTS', 'A_SHOTS_ON_TARGET',
       'A_DEEP', 'A_PPDA', 'A_XPTS',  'GAMEURL_y', 'ID','last_game_minute',
       'start_min_game']].head(3)

,H_DATETIME,H_GAMEINFO,H_TEAMNAMES,H_GOALS,H_BALL_POSSESSION,H_GOAL_ATTEMPTS,H_SHOTS_ON_GOAL,H_SHOTS_OFF_GOAL,H_BLOCKED_SHOTS,H_FREE_KICKS,H_CORNER_KICKS,H_OFFSIDES,H_GOALKEEPER_SAVES,H_FOULS,H_YELLOW_CARDS,H_RED_CARDS,H_TOTAL_PASSES,H_COMPLETED_PASSES,H_TACKLES,H_ATTACKS,H_DANGEROUS_ATTACKS,A_DATETIME,A_GAMEINFO,A_TEAMNAMES,A_GOALS,A_BALL_POSSESSION,A_GOAL_ATTEMPTS,A_SHOTS_ON_GOAL,A_SHOTS_OFF_GOAL,A_BLOCKED_SHOTS,A_FREE_KICKS,A_CORNER_KICKS,A_OFFSIDES,A_GOALKEEPER_SAVES,A_FOULS,A_YELLOW_CARDS,A_RED_CARDS,A_TOTAL_PASSES,A_COMPLETED_PASSES,A_TACKLES,A_ATTACKS,A_DANGEROUS_ATTACKS,H_CAT,GAMEURL_x,H_INTERCEPTIONS,A_EXPECTED_GOALS_XG,H_THROWINS,H_EXPECTED_GOALS_XG,A_THROWINS,A_INTERCEPTIONS,H_CLEARANCES_TOTAL,H_SHOTS_INSIDE_THE_BOX,A_PASSES_IN_THE_FINAL_THIRD,A_CLEARANCES_TOTAL,H_CROSSES,H_PASSES_IN_THE_FINAL_THIRD,A_CROSSES,A_TOUCHES_IN_THE_OPPOSITION_BOX,H_SHOTS_OUTSIDE_THE_BOX,H_TOUCHES_IN_THE_OPPOSITION_BOX,H_BIG_CHANCES,A_BIG_CHANCES,H_HIT_THE_WOODWORK,A_SHOTS_OUTSIDE_THE_BOX,A_SHOTS_INSIDE_THE_BOX,A_HIT_THE_WOODWORK,H_HEADED_GOALS,A_HEADED_GOALS,H_GOALS_COMPLETE_GAME,XG,SHOTS,SHOTS_ON_TARGET,DEEP,PPDA,XPTS,A_GOALS_COMPLETE_GAME,A_XG,A_SHOTS,A_SHOTS_ON_TARGET,A_DEEP,A_PPDA,A_XPTS,GAMEURL_y,ID,last_game_minute,start_min_game
0,02.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,Wolfsburg,0,68,10,3,2,5.0,5,5,0,0,3,NaN,NaN,341,302,NaN,NaN,NaN,02.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,Augsburg,1,32,2,1,1,0.0,3,0,0,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cat123,https://www.flashscore.com/match/AoSdOZaB/#/ma...,1,0.31,10,0.35,7,3,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,1,1.1,18,6,14,8.4,1.9,1,0.5,3,1,1,18.7,0.8,https://understat.com/match/27820,75,90,0
1,02.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,Wolfsburg,0,72,8,3,3,2.0,9,3,0,0,4,1.0,NaN,285,231,NaN,NaN,NaN,02.11.2024 14:30,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,Augsburg,1,28,1,0,1,0.0,4,1,1,2,9,3.0,NaN,NaN,NaN,NaN,NaN,NaN,cat124,https://www.flashscore.com/match/AoSdOZaB/#/ma...,4,0.05,12,0.95,13,7,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,1,1.1,18,6,14,8.4,1.9,1,0.5,3,1,1,18.7,0.8,https://understat.com/match/27820,75,90,0
2,24.08.2024 15:30,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 1,Augsburg,0,49,4,0,4,NaN,3,4,1,2,8,3.0,NaN,182,142,NaN,45.0,30.0,24.08.2024 15:30,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 1,Werder Bremen,1,51,5,3,2,NaN,9,2,0,0,3,1.0,NaN,179.0,124.0,NaN,49.0,15.0,cat10,https://www.flashscore.com/match/YZZcMwBs/#/ma...,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,2,1.3,11,3,3,14.8,1.2,2,1.5,11,5,11,10.8,1.5,https://understat.com/match/27746,7,90,0


In [ ]:
dfxg_df_merged_cleaned = df_cleaning_converting(df)
dfxg_df_merged_cleaned.head(3)

error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
error?
!!! Index(['H_TEAMNAMES', 'A_TEAMNAMES', 'H_GOALS', 'A_GOALS', 'H_BALL_POSSESSION',
       'A_BALL_POSSESSION', 'H_GOAL_ATTEMPTS', 'A_GOAL_ATTEMPTS',
       'H_SHOTS_ON_GOAL', 'A_SHOTS_ON_GOAL', 'H_SHOTS_OFF_GOAL',
       'A_SHOTS_OFF_GOAL', 'H_FREE_KICKS', 'H_RED_CARDS', 'A_RED_CARDS',
       'A_FREE_KICKS', 'H_CORNER_KICKS', 'A_CORNER_KICKS', 'H_OFFSIDES',
       'A_OFFSIDES', 'H_GOALKEEPER_SAVES', 'A_GOALKEEPER_SAVES', 'H_FOULS',
       'A_FOULS', 'A_GAMEINFO', 'A_DATETIME', 'XG', 'H_GOALS_COMPLETE_GAME',
       'XPTS', 'A_XG', 'A_GOALS_COMPLETE_GAME', 'A_XPTS', 'TIMING_CHART_XG',
       'HOMEXG_COMPLETE_GAME', 'AWAYXG_COMPLETE_GAME', 'R', 'xg_halftime',
       'Axg_halftime'],
      dtype='object')


,Home,Opponent,R,G-H,G-A,BP-H,BP-A,GA-H,GA-A,SoG-H,SoG-A,SoffG-H,SoffG-A,FK-H,H_Red Cards,A_Red Cards,FK-A,C-H,C-A,Off-H,Off-A,GoKeSa-H,GoKeSa-A,F-H,F-A,Round,Date,IsHome,xG,H_GOALS_COMPLETE_GAME,xPTS,A_xG,A_GOALS_COMPLETE_GAME,A_xPTS,xg_halftime,Axg_halftime,homexg_complete_game,awayxg_complete_game
0,Wolfsburg,Augsburg,X,0,1,68,32,10,2,3,1,2,1,5,NaN,NaN,3,5,0,0,0,0,3,3,5,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,02.11.2024 14:30,0,1.1,1,1.9,0.5,1,0.8,0.4,0.4,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.01, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0..."
1,Wolfsburg,Augsburg,X,0,1,72,28,8,1,3,0,3,1,9,NaN,NaN,4,3,1,0,1,0,2,4,9,FOOTBALL\nGERMANY: BUNDESLIGA - ROUND 9,02.11.2024 14:30,0,1.1,1,1.9,0.5,1,0.8,0.4,0.4,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.01, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0..."
2,Augsburg,Werder Bremen,X,0,1,49,51,4,5,0,3,4,2,3,NaN,NaN,9,4,2,1,0,2,0,8,3,FOOTBALL\r\nGERMANY: BUNDESLIGA - ROUND 1,24.08.2024 15:30,0,1.3,2,1.2,1.5,2,1.5,1.1,0.6,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0..."


In [21]:
df4Home, df4OpponentReversed = df_specific_team(
    dfxg_df_merged_cleaned, team)

df4Complete = create_df4Complete(df4Home, df4OpponentReversed)
slidertext = 'Show last x halftimes'
nrGames = st.sidebar.slider(slidertext, max_value=len(
    df4Complete), value=len(df4Complete), step=2)

# change rows of df depending on userinput
df4Complete = df4Complete[:nrGames]
df4Complete = df4Complete.sort_values("Date", ascending=False)
df4Complete = df4Complete.round(1)

df4Complete[['xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A', 'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS',  "A_Red Cards", "H_Red Cards"]] = df4Complete[[
                'xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A', 'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS',  "A_Red Cards", "H_Red Cards"]].apply(pd.to_numeric, errors='coerce', axis=1)

df4Complete['A_Red Cards'] = df4Complete['A_Red Cards'].fillna(0)
df4Complete['H_Red Cards'] = df4Complete['H_Red Cards'].fillna(0)

df4Complete_show = df4Complete[['Home', 'Opponent', 'IsHome', 'R', 'xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                                'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS', 'Date', 'xg_halftime', 'Axg_halftime', "A_Red Cards", "H_Red Cards"]]


2025-01-11 16:58:34.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:34.649 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:34.650 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:34.652 No runtime found, using MemoryCacheStorageManager
2025-01-11 16:58:34.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:34.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:34.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:34.672 No runtime found, using MemoryCacheStorageManager
2025-01-11 16:58:34.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

df4Opponent.columns before reassignment oppo Index(['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
       'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H', 'H_Red Cards',
       'A_Red Cards', 'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H',
       'GoKeSa-A', 'F-H', 'F-A', 'Round', 'Date', 'IsHome', 'xG',
       'H_GOALS_COMPLETE_GAME', 'xPTS', 'A_xG', 'A_GOALS_COMPLETE_GAME',
       'A_xPTS', 'xg_halftime', 'Axg_halftime', 'homexg_complete_game',
       'awayxg_complete_game', '1x2'],
      dtype='object')
df4OpponentReversed.columns after reassignment oppo Index(['Home', 'Opponent', '1x2', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H',
       'GA-A', 'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H', 'H_Red Cards',
       'A_Red Cards', 'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H',
       'GoKeSa-A', 'F-H', 'F-A', 'Round', 'Date', 'IsHome', 'xG', 'xPTS',
       'H_GOALS_COMPLETE_GAME', 'A_xG', 'A_xPTS', 'A_GOALS_COMPLETE_GAME',
       'xg_halftime', 'Axg_half

2025-01-11 16:58:35.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [22]:

# calc the xg per minute over all games to get the mean over all minutes from all games!

values = st.sidebar.slider(
    'BP-Range for xG per minute',
    0.0, 100.0, (0.0, 100.0))

smaller_bp = values[1]
bigger_bp = values[0]
dfxg_homexg_complete_game = pd.DataFrame(df4Complete[(df4Complete["BP-H"]>bigger_bp) & (df4Complete["BP-H"]<smaller_bp)].homexg_complete_game.tolist(), index= df4Complete[(df4Complete["BP-H"]>bigger_bp) & (df4Complete["BP-H"]<smaller_bp)].index)
dfxg_awayxg_complete_game = pd.DataFrame(df4Complete[(df4Complete["BP-H"]>bigger_bp) & (df4Complete["BP-H"]<smaller_bp)].awayxg_complete_game.tolist(), index= df4Complete[(df4Complete["BP-H"]>bigger_bp) & (df4Complete["BP-H"]<smaller_bp)].index)
dfxg_homexg_complete_game = dfxg_homexg_complete_game.apply(pd.to_numeric)
dfxg_awayxg_complete_game = dfxg_awayxg_complete_game.apply(pd.to_numeric)
dfxg_homexg_complete_game = dfxg_homexg_complete_game.fillna(0)
dfxg_awayxg_complete_game = dfxg_awayxg_complete_game.fillna(0)
if len(dfxg_homexg_complete_game) > 0 and len(dfxg_awayxg_complete_game) > 0:
    dfxg_homexg_complete_game = dfxg_homexg_complete_game.diff(axis=1)
    dfxg_awayxg_complete_game = dfxg_awayxg_complete_game.diff(axis=1)
    dfxg_homexg_complete_game[dfxg_homexg_complete_game < 0] = 0
    dfxg_awayxg_complete_game[dfxg_awayxg_complete_game < 0] = 0

dfxg_homexg_complete_game_all_bps = pd.DataFrame(df4Complete.homexg_complete_game.tolist(), index= df4Complete.index)
dfxg_awayxg_complete_game_all_bps = pd.DataFrame(df4Complete.awayxg_complete_game.tolist(), index= df4Complete.index)
dfxg_homexg_complete_game_all_bps = dfxg_homexg_complete_game_all_bps.apply(pd.to_numeric)
dfxg_awayxg_complete_game_all_bps = dfxg_awayxg_complete_game_all_bps.apply(pd.to_numeric)
dfxg_homexg_complete_game_all_bps = dfxg_homexg_complete_game_all_bps.fillna(0)
dfxg_awayxg_complete_game_all_bps = dfxg_awayxg_complete_game_all_bps.fillna(0)
if len(dfxg_homexg_complete_game_all_bps) > 0 and len(dfxg_awayxg_complete_game_all_bps) > 0:
    dfxg_homexg_complete_game_all_bps = dfxg_homexg_complete_game_all_bps.diff(axis=1)
    dfxg_awayxg_complete_game_all_bps = dfxg_awayxg_complete_game_all_bps.diff(axis=1)
    dfxg_homexg_complete_game_all_bps[dfxg_homexg_complete_game_all_bps < 0] = 0
    dfxg_awayxg_complete_game_all_bps[dfxg_awayxg_complete_game_all_bps < 0] = 0

dfxg_homexg_complete_game_bigger_55 = pd.DataFrame(df4Complete[(df4Complete["BP-H"]>55)].homexg_complete_game.tolist(), index= df4Complete[(df4Complete["BP-H"]>55)].index)
dfxg_awayxg_complete_game_bigger_55 = pd.DataFrame(df4Complete[(df4Complete["BP-H"]>55)].awayxg_complete_game.tolist(), index= df4Complete[(df4Complete["BP-H"]>55)].index)
dfxg_homexg_complete_game_bigger_55 = dfxg_homexg_complete_game_bigger_55.apply(pd.to_numeric)
dfxg_awayxg_complete_game_bigger_55 = dfxg_awayxg_complete_game_bigger_55.apply(pd.to_numeric)
dfxg_homexg_complete_game_bigger_55 = dfxg_homexg_complete_game_bigger_55.fillna(0)
dfxg_awayxg_complete_game_bigger_55 = dfxg_awayxg_complete_game_bigger_55.fillna(0)
if len(dfxg_homexg_complete_game_bigger_55) > 0 and len(dfxg_awayxg_complete_game_bigger_55) > 0:
    dfxg_homexg_complete_game_bigger_55 = dfxg_homexg_complete_game_bigger_55.diff(axis=1)
    dfxg_awayxg_complete_game_bigger_55 = dfxg_awayxg_complete_game_bigger_55.diff(axis=1)
    dfxg_homexg_complete_game_bigger_55[dfxg_homexg_complete_game_bigger_55 < 0] = 0
    dfxg_awayxg_complete_game_bigger_55[dfxg_awayxg_complete_game_bigger_55 < 0] = 0

dfxg_homexg_complete_game_smaller_45 = pd.DataFrame(df4Complete[(df4Complete["BP-H"]<45)].homexg_complete_game.tolist(), index= df4Complete[(df4Complete["BP-H"]<45)].index)
dfxg_awayxg_complete_game_smaller_45 = pd.DataFrame(df4Complete[(df4Complete["BP-H"]<45)].awayxg_complete_game.tolist(), index= df4Complete[(df4Complete["BP-H"]<45)].index)
dfxg_homexg_complete_game_smaller_45 = dfxg_homexg_complete_game_smaller_45.apply(pd.to_numeric)
dfxg_awayxg_complete_game_smaller_45 = dfxg_awayxg_complete_game_smaller_45.apply(pd.to_numeric)
dfxg_homexg_complete_game_smaller_45 = dfxg_homexg_complete_game_smaller_45.fillna(0)
dfxg_awayxg_complete_game_smaller_45 = dfxg_awayxg_complete_game_smaller_45.fillna(0)
if len(dfxg_homexg_complete_game_smaller_45) > 0 and len(dfxg_awayxg_complete_game_smaller_45) > 0:
    dfxg_homexg_complete_game_smaller_45 = dfxg_homexg_complete_game_smaller_45.diff(axis=1)
    dfxg_awayxg_complete_game_smaller_45 = dfxg_awayxg_complete_game_smaller_45.diff(axis=1)
    dfxg_homexg_complete_game_smaller_45[dfxg_homexg_complete_game_smaller_45 < 0] = 0
    dfxg_awayxg_complete_game_smaller_45[dfxg_awayxg_complete_game_smaller_45 < 0] = 0


print("df4Complete_show")
print(df4Complete_show)
   

2025-01-11 16:58:35.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.098 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-11 16:58:35.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


df4Complete_show
        Home            Opponent  IsHome  R  xG  A_xG  G-H  G-A  BP-H  BP-A  \
0   Augsburg           Wolfsburg       0  X 0.5   1.1  1.0  0.0  32.0  68.0   
1   Augsburg           Wolfsburg       0  X 0.5   1.1  1.0  0.0  28.0  72.0   
8   Augsburg   Borussia Dortmund       1  X 0.6   0.8  1.0  1.0  18.0  82.0   
9   Augsburg   Borussia Dortmund       1  X 0.6   0.8  1.0  1.0  28.0  72.0   
6   Augsburg            Freiburg       0  X 0.9   1.1  0.0  3.0  35.0  65.0   
7   Augsburg            Freiburg       0  X 0.9   1.1  0.0  3.0  51.0  49.0   
14  Augsburg  B. Monchengladbach       1  X 1.8   0.8  1.0  1.0  44.0  56.0   
15  Augsburg  B. Monchengladbach       1  X 1.8   0.8  1.0  0.0  34.0  66.0   
4   Augsburg          RB Leipzig       0  X 1.1   2.8  0.0  2.0  42.0  58.0   
5   Augsburg          RB Leipzig       0  X 1.1   2.8  0.0  2.0  44.0  56.0   
12  Augsburg               Mainz       1  X 2.8   1.3  1.0  1.0  68.0  32.0   
13  Augsburg               Mainz   

In [23]:
asdasd

NameError: name 'asdasd' is not defined

In [ ]:
 # create df for visualizing
df4CompleteGraph = df4Complete.copy()


teamname_to_search = st.sidebar.text_input("Search for Opponent", )
df4CompleteGraph = df4CompleteGraph[df4CompleteGraph["Opponent"].str.contains("{}".format(teamname_to_search), na=False, case=False)]


df4CompleteGraph["SoG-H-SoG-A"] = df4CompleteGraph["SoG-H"] - \
    df4CompleteGraph["SoG-A"]
df4CompleteGraph["SoG-H-SoG-A"] = df4CompleteGraph["SoG-H-SoG-A"].clip(
    lower=0)

df4CompleteGraph["SoG-A-SoG-H"] = df4CompleteGraph["SoG-A"] - \
    df4CompleteGraph["SoG-H"]
df4CompleteGraph["SoG-A-SoG-H"] = df4CompleteGraph["SoG-A-SoG-H"].clip(
    lower=0)

df4CompleteGraph.sort_values("IsHome", ascending=False)


# calculate the difference of team xg vs oppo xg
dfxg_complete_game = dfxg_homexg_complete_game.clip(lower=0).mean() - dfxg_awayxg_complete_game.clip(lower=0).mean()
fig_xg_perminute_home = go.Figure(data=[
    go.Bar(
        name='xG', y=dfxg_homexg_complete_game.clip(lower=0).mean(), marker_color='green'
        ),
    # -1 to show the bars to the below instead of above!
    go.Bar(
        name='Opponent xG', y=dfxg_awayxg_complete_game.clip(lower=0).mean()*-1, marker_color='red' 
        ),
])
fig_xg_perminute_home.add_trace(
    go.Scatter(
        name='Diff xG',
        y=dfxg_complete_game,
        line=dict(color='gold', width=3)
        )
    )
fig_xg_perminute_home.update_layout(
    title_text='Expectedgoals per minute: {} < bp < {}'.format(int(bigger_bp), int(smaller_bp)), title_x=title_x,
    yaxis=dict(
        title="xG"
    ),
    autosize=False,
    width=1400,
    height=400,
    )
fig_xg_perminute_home.update_yaxes(range=[(dfxg_awayxg_complete_game.mean().max()+0.02)*-1, dfxg_homexg_complete_game.mean().max()+0.02])

# calculate the difference of team xg vs oppo xg
dfxg_complete_game_all_bps = dfxg_homexg_complete_game_all_bps.clip(lower=0).mean() - dfxg_awayxg_complete_game_all_bps.clip(lower=0).mean()
fig_xg_homexg_complete_game_all_bpse = go.Figure(data=[
    go.Bar(
        name='xG', y=dfxg_homexg_complete_game_all_bps.clip(lower=0).mean(), marker_color='green'
        ),
    # -1 to show the bars to the below instead of above!
    go.Bar(
        name='Opponent xG', y=dfxg_awayxg_complete_game_all_bps.clip(lower=0).mean()*-1, marker_color='red' 
        ),
])
fig_xg_homexg_complete_game_all_bpse.add_trace(
    go.Scatter(
        name='Diff xG',
        y=dfxg_complete_game_all_bps,
        line=dict(color='gold', width=3)
        )
    )
fig_xg_homexg_complete_game_all_bpse.update_layout(
    title_text='Expectedgoals per minute', title_x=title_x,
    yaxis=dict(
        title="xG"
    ),
    autosize=False,
    width=1400,
    height=400,
    )
fig_xg_homexg_complete_game_all_bpse.update_yaxes(range=[(dfxg_awayxg_complete_game.mean().max()+0.02)*-1, dfxg_homexg_complete_game.mean().max()+0.02])



# calculate the difference of team xg vs oppo xg
dfxg_complete_game_bigger_55 = dfxg_homexg_complete_game_bigger_55.clip(lower=0).mean() - dfxg_awayxg_complete_game_bigger_55.clip(lower=0).mean()
fig_xg_perminute_home_bigger_55 = go.Figure(data=[
    go.Bar(
        name='xG', y=dfxg_homexg_complete_game_bigger_55.clip(lower=0).mean(), marker_color='green'
        ),
    # -1 to show the bars to the below instead of above!
    go.Bar(
        name='Opponent xG', y=dfxg_awayxg_complete_game_bigger_55.clip(lower=0).mean()*-1, marker_color='red' 
        ),
])
fig_xg_perminute_home_bigger_55.add_trace(
    go.Scatter(
        name='Diff xG',
        y=dfxg_complete_game_bigger_55,
        line=dict(color='gold', width=3)
        )
    )
fig_xg_perminute_home_bigger_55.update_layout(
    title_text='Expectedgoals per minute: bp > 55', title_x=title_x,
    yaxis=dict(
        title="xG"
    ),
    autosize=False,
    width=1400,
    height=400,
    )
fig_xg_perminute_home_bigger_55.update_yaxes(range=[(dfxg_awayxg_complete_game.mean().max()+0.02)*-1, dfxg_homexg_complete_game.mean().max()+0.02])



# calculate the difference of team xg vs oppo xg
dfxg_complete_game_smaller_45 = dfxg_homexg_complete_game_smaller_45.clip(lower=0).mean() - dfxg_awayxg_complete_game_smaller_45.clip(lower=0).mean()
fig_xg_perminute_home_smaller_45 = go.Figure(data=[
    go.Bar(
        name='xG', y=dfxg_homexg_complete_game_smaller_45.clip(lower=0).mean(), marker_color='green'
        ),
    # -1 to show the bars to the below instead of above!
    go.Bar(
        name='Opponent xG', y=dfxg_awayxg_complete_game_smaller_45.clip(lower=0).mean()*-1, marker_color='red' 
        ),
])
fig_xg_perminute_home_smaller_45.add_trace(
    go.Scatter(
        name='Diff xG',
        y=dfxg_complete_game_smaller_45,
        line=dict(color='gold', width=3)
        )
    )
fig_xg_perminute_home_smaller_45.update_layout(
    title_text='Expectedgoals per minute: bp < 45', title_x=title_x,
    yaxis=dict(
        title="xG"
    ),
    autosize=False,
    width=1400,
    height=400,
    )
fig_xg_perminute_home_smaller_45.update_yaxes(range=[(dfxg_awayxg_complete_game.mean().max()+0.02)*-1, dfxg_homexg_complete_game.mean().max()+0.02])


figScatter_SoG_SoGA = px.scatter(
    df4CompleteGraph.sort_values("IsHome", ascending=False),  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    symbol = 'IsHome',
    symbol_sequence= ['diamond-cross', 'diamond'],
    size="SoG-H-SoG-A",
    text="Opponent",
    width=widthfig,
).update_traces(textposition='top center', selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray"
).update_traces(
    marker=dict(color='green'), selector={'type': 'histogram'}
)
figScatter_SoG_SoGA.update_xaxes(range=[5, 95])
figScatter_SoG_SoGA.update_layout(
    title_text='All halftimes: Shots on Goal - Shots on Goal Opponent', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figScatter_SoG_SoGA.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))

figScatter_SoGA_soG = px.scatter(
    df4CompleteGraph.sort_values("IsHome", ascending=False),  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="SoG-A-SoG-H",
    symbol = 'IsHome',
    symbol_sequence= ['circle-x', 'circle'],
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    title="SoGA-SoGH - Halftimes",
    # color_discrete_map={"W": "green", "D": "gray", "L": "red"}
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(
    textposition='top center', selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray").update_traces(
        marker=dict(color='red'), selector={'type': 'histogram'}
)
figScatter_SoGA_soG.update_xaxes(range=[5, 95])
figScatter_SoGA_soG.update_layout(
    title_text='All halftimes: Shots on Goal Opponent - Shots on Goal', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    )
)
figScatter_SoGA_soG.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))


# delete games where there is no two halftimes!
df4CompleteGraph = df4CompleteGraph[df4CompleteGraph.groupby(
    'Opponent')['Opponent'].transform('size') >= 2]
df4CompleteGraph = df4CompleteGraph.sort_index()
# second half is second entry always!
df4CompleteGraph["halftime"] = "0"
df4CompleteGraph['halftime'] = np.where(df4CompleteGraph.index % 2, '1', '2')

figScatter_h1_soG_SoGA = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"].sort_values("IsHome", ascending=False),
    x='BP-H',
    y='GoalDiff',
    size="SoG-H-SoG-A",
    text="Opponent",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    symbol = 'IsHome',
    symbol_sequence= ['diamond-cross', 'diamond'],
    width=widthfig,
    marginal_x="histogram",
).update_traces(textposition='top center',  marker=dict(
    color='green'), selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray"
).update_traces(marker=dict(
    color='green'), selector={'type': 'histogram'}
)
figScatter_h1_soG_SoGA.update_xaxes(range=[5, 95])
figScatter_h1_soG_SoGA.update_layout(
    title_text='Halftime 1: Shots on Goal - Shots on Goal Opponent', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figScatter_h1_soG_SoGA.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))

figScatter_h1_SoGA_soG = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"].sort_values("IsHome", ascending=False),
    x='BP-H',
    y='GoalDiff',
    size="SoG-A-SoG-H",
    text="Opponent",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    symbol = 'IsHome',
    symbol_sequence= ['circle-x', 'circle'],
    width=widthfig,
    marginal_x="histogram",
).update_traces(textposition='top center', marker=dict(
    color='red'), selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray"
).update_traces(marker=dict(
    color='red'), selector={'type': 'histogram'}
)
figScatter_h1_SoGA_soG.update_xaxes(range=[5, 95])
figScatter_h1_SoGA_soG.update_layout(
    title_text='Halftime 1: Shots on Goal Opponent - Shots on Goal', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figScatter_h1_SoGA_soG.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))


figScatter_h2_SoG_SoGA = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"].sort_values("IsHome", ascending=False),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="SoG-H-SoG-A",
    symbol = 'IsHome',
    symbol_sequence= ['diamond-cross', 'diamond'],
    text="Opponent",
    width=widthfig,
).update_traces(textposition='top center',  marker=dict(
    color='green'), selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray"
).update_traces(marker=dict(
    color='green'), selector={'type': 'histogram'}
)
figScatter_h2_SoG_SoGA.update_xaxes(range=[5, 95])
figScatter_h2_SoG_SoGA.update_layout(
    title_text='Halftime 2: Shots on Goal - Shots on Goal Opponent', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    )
)
figScatter_h2_SoG_SoGA.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))


figScatter_h2_SoGA_SoG = px.scatter(
    # .query(f'Date.between{end_date}'),
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"].sort_values("IsHome", ascending=False),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="SoG-A-SoG-H",
    symbol = 'IsHome',
    symbol_sequence= ['circle-x', 'circle'],
    text="Opponent",
    width=widthfig,
).update_traces(textposition='top center', marker=dict(
    color='red'), selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray"
).update_traces(marker=dict(
    color='red'), selector={'type': 'histogram'}
)
figScatter_h2_SoGA_SoG.update_xaxes(range=[5, 95])
figScatter_h2_SoGA_SoG.update_layout(
    title_text='Halftime 2: Shots on Goal Opponent - Shots on Goal', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figScatter_h2_SoGA_SoG.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))

naming1x2 = {"W": "Win", "D": "Draw", "L": "Loss"}
df4CompleteGraph['Halftime result'] = df4CompleteGraph['1x2'].replace(
    naming1x2)

try:
    highest_count_yaxis = df4CompleteGraph.groupby(["BPTypes", "halftime"]).agg(
        'count').sort_values("Opponent", ascending=False).iloc[0].Home
except:
    highest_count_yaxis = 0

df4CompleteGraph['count'] = 1
df4CompleteGraph_ht1 = df4CompleteGraph[df4CompleteGraph["halftime"] == "1"]
df4CompleteGraph_ht2 = df4CompleteGraph[df4CompleteGraph["halftime"] == "2"]
required_categories = ['<45', '45-55', '>55']
for category in required_categories:
    if category not in df4CompleteGraph_ht1['BPTypes'].unique():
        # Append a row with the missing category and a count of 0
        df4CompleteGraph = df4CompleteGraph.append({
            'BPTypes': category, 
            'Halftime result': 'Draw',  
            'Home': '',  
            'Opponent': '',  
            'IsHome': 0, 
            'R': '',  
            'xG': 0.0,  
            'halftime': '1',
            'A_xG': 0.0,
            'count': 0
        }, ignore_index=True)
    if category not in df4CompleteGraph_ht2['BPTypes'].unique():
        # Append a row with the missing category and a count of 0
        df4CompleteGraph = df4CompleteGraph.append({
            'BPTypes': category, 
            'Halftime result': 'Draw',  
            'Home': '',  
            'Opponent': '',  
            'IsHome': 0, 
            'R': '',  
            'xG': 0.0,  
            'halftime': '2',
            'A_xG': 0.0,
            'count': 0
        }, ignore_index=True)


# Create data for histogram 2
BarBallpossesionstylesResultsHalftime1 = px.bar(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],
    x='BPTypes',
    y='count',
    # text=df4CompleteGraph.index,
    color='Halftime result',
    color_discrete_map={"Win": "green", "Draw": "gray", "Loss": "red"},
    width=widthfig,
    # height=heightfig,
    text="Opponent",
)
BarBallpossesionstylesResultsHalftime1.update_xaxes(categoryorder="array", categoryarray=['<45', '45-55', '>55'],).update_yaxes(
    range=[0, highest_count_yaxis])

BarBallpossesionstylesResultsHalftime1.update_layout(
    title_text='Ballpossesionstyles - results halftime 1', title_x=title_x, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)

# Create data for histogram 2
BarBallpossesionstylesResultsHalftime2 = px.bar(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],
    x='BPTypes',
    y='count',
    # text=df4CompleteGraph.index,
    color='Halftime result',
    color_discrete_map={"Win": "green", "Draw": "gray", "Loss": "red"},
    width=widthfig,
    # height=heightfig,
    text="Opponent",
)
BarBallpossesionstylesResultsHalftime2.update_xaxes(categoryorder="array", categoryarray=['<45', '45-55', '>55']).update_yaxes(
    range=[0, highest_count_yaxis])
BarBallpossesionstylesResultsHalftime2.update_layout(
    title_text='Ballpossesionstyles - results halftime 2', title_x=title_x, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)

print(df4CompleteGraph.columns)
print(df4CompleteGraph[["xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])

df4CompleteGraph.xg_halftime = df4CompleteGraph.xg_halftime.astype(float).fillna(0.0)
df4CompleteGraph.Axg_halftime = df4CompleteGraph.Axg_halftime.astype(float).fillna(0.0) 

# all xg values for both halftimes!
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG"] - df4CompleteGraph["A_xG"]
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG"] - df4CompleteGraph["xG"]
# all values for first half
df4CompleteGraph["xg_halftime-Axg_halftime"] = df4CompleteGraph["xg_halftime"] - df4CompleteGraph["Axg_halftime"]
df4CompleteGraph["Axg_halftime-xg_halftime"] = df4CompleteGraph["Axg_halftime"] - df4CompleteGraph["xg_halftime"]

# all values for second half
df4CompleteGraph["diff_xg_fulltime-diff_xg_halftime"] = df4CompleteGraph["xG-A_xG"] - df4CompleteGraph["xg_halftime-Axg_halftime"]
df4CompleteGraph["diff_Axg_fulltime-diff_Axg_halftime"] = df4CompleteGraph["A_xG-xG"] - df4CompleteGraph["Axg_halftime-xg_halftime"]

# formatting
df4CompleteGraph["diff_xg_fulltime-diff_xg_halftime"] = df4CompleteGraph["diff_xg_fulltime-diff_xg_halftime"].clip(lower=0)
df4CompleteGraph["diff_xg_fulltime-diff_xg_halftime"] = df4CompleteGraph["diff_xg_fulltime-diff_xg_halftime"].round(2)
df4CompleteGraph["diff_Axg_fulltime-diff_Axg_halftime"] = df4CompleteGraph["diff_Axg_fulltime-diff_Axg_halftime"].clip(lower=0)
df4CompleteGraph["diff_Axg_fulltime-diff_Axg_halftime"] = df4CompleteGraph["diff_Axg_fulltime-diff_Axg_halftime"].round(2)
df4CompleteGraph["xg_halftime-Axg_halftime"] = df4CompleteGraph["xg_halftime-Axg_halftime"].clip(lower=0)
df4CompleteGraph["xg_halftime-Axg_halftime"] = df4CompleteGraph["xg_halftime-Axg_halftime"].round(2)
df4CompleteGraph["Axg_halftime-xg_halftime"] = df4CompleteGraph["Axg_halftime-xg_halftime"].clip(lower=0)
df4CompleteGraph["Axg_halftime-xg_halftime"] = df4CompleteGraph["Axg_halftime-xg_halftime"].round(2)
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG-A_xG"].clip(lower=0)
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG-A_xG"].round(2)
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG-xG"].clip(lower=0)
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG-xG"].round(2)


print(df4CompleteGraph[["xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])
ht1 = df4CompleteGraph[df4CompleteGraph["halftime"] == "1"]
print(ht1[["IsHome","xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])
ht2 = df4CompleteGraph[df4CompleteGraph["halftime"] == "2"]
print(ht2[["IsHome","xG","A_xG", "xg_halftime", "Axg_halftime","halftime","Opponent",'Halftime result',"timestamp"]])

# Create barchart for xg per bptypes 1
BarBallpossesionstylesXGHalftime1 = px.bar(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],
    x='BPTypes',
    y=['xg_halftime-Axg_halftime', 'Axg_halftime-xg_halftime'],
    barmode='group',
    # text=df4CompleteGraph.index,
    # title="BP-Styles - Halftimes",
    # color='Halftime result',
    color_discrete_map={"xg_halftime-Axg_halftime": "green", "Axg_halftime-xg_halftime": "red"},
    width=widthfig,
    # height=heightfig,
    # opacity=0.5,
    text="Opponent",
).update_xaxes(categoryorder="array", categoryarray=['<45', '45-55', '>55']).update_yaxes(
    range=[0, highest_count_yaxis])
BarBallpossesionstylesXGHalftime1.update_layout(
    title_text='Ballpossesionstyles - xG halftime 1', title_x=title_x, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)
BarBallpossesionstylesXGHalftime1.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))


# Create barchart for xg per bptypes 2
BarBallpossesionstylesXGHalftime2 = px.bar(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],
    x='BPTypes',
    y=['diff_xg_fulltime-diff_xg_halftime', 'diff_Axg_fulltime-diff_Axg_halftime'],
    barmode='group',
    # text=df4CompleteGraph.index,
    # title="BP-Styles - Halftimes",
    # color='Halftime result',
    color_discrete_map={"diff_xg_fulltime-diff_xg_halftime": "green", "diff_Axg_fulltime-diff_Axg_halftime": "red"},
    width=widthfig,
    # height=heightfig,
    # opacity=0.5,
    text="Opponent",
).update_xaxes(categoryorder="array", categoryarray=['<45', '45-55', '>55']).update_yaxes(
    range=[0, highest_count_yaxis])
BarBallpossesionstylesXGHalftime2.update_layout(
    title_text='Ballpossesionstyles - xG halftime 2', title_x=title_x, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)
BarBallpossesionstylesXGHalftime2.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))

figHistogramxG_A_xG_1Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="xg_halftime-Axg_halftime",
    symbol = 'IsHome',
    symbol_sequence= ['diamond-cross', 'diamond'],
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray" ).update_traces(
    marker=dict(color='green'), selector={'type': 'histogram'}
)
figHistogramxG_A_xG_1Ht.update_xaxes(range=[5, 95])
figHistogramxG_A_xG_1Ht.update_layout(
    title_text='Ht1: Expectedgoals - Expectedgoals Opponent', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figHistogramxG_A_xG_1Ht.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))

figHistogramA_xG_xG_1Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "1"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="Axg_halftime-xg_halftime",
    symbol = 'IsHome',
    symbol_sequence= ['circle-x', 'circle'],
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', selector={'type': 'scatter'}, textfont_size=9, textfont_color="gray" ).update_traces(
    marker=dict(color='red'), selector={'type': 'histogram'}
)
figHistogramA_xG_xG_1Ht.update_xaxes(range=[5, 95])
figHistogramA_xG_xG_1Ht.update_layout(
    title_text='Ht1: Expectedgoals Opponent - Expectedgoals', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figHistogramA_xG_xG_1Ht.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))


# Determine if there was a red card in a game
df4CompleteGraph['red_card'] = df4CompleteGraph[['H_Red Cards', 'A_Red Cards']].apply(lambda x: x[0] > 0 or x[1] > 0, axis=1)
# Define marker properties based on the presence of a red card
df4CompleteGraph['marker_properties'] = df4CompleteGraph['red_card'].apply(
    lambda x: {'line_width': 2, 'line_color': 'red'} if x else {'line_width': 1, 'line_color': 'black'}
)

figHistogramxG_A_xG_2Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="diff_xg_fulltime-diff_xg_halftime",
    symbol = 'IsHome',
    symbol_sequence= ['diamond-cross', 'diamond'],
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', selector={'type': 'scatter'},textfont_size=9, textfont_color="gray" ).update_traces(
    marker=dict(color='green'), selector={'type': 'histogram'}
)
figHistogramxG_A_xG_2Ht.update_xaxes(range=[5, 95])
figHistogramxG_A_xG_2Ht.update_layout(
    title_text='Ht2: Expectedgoals - Expectedgoals Opponent', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figHistogramxG_A_xG_2Ht.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))

figHistogramA_xG_xG_2Ht = px.scatter(
    df4CompleteGraph[df4CompleteGraph["halftime"] == "2"],  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    marginal_x="histogram",
    color="timestamp",
    hover_data=['H_Red Cards', 'A_Red Cards', 'Date'],
    size="diff_Axg_fulltime-diff_Axg_halftime",
    symbol = 'IsHome',
    symbol_sequence= ['circle-x', 'circle'],
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', selector={'type': 'scatter'},textfont_size=9, textfont_color="gray" ).update_traces(
    marker=dict(color='red'), selector={'type': 'histogram'}
)
figHistogramA_xG_xG_2Ht.update_xaxes(range=[5, 95])
figHistogramA_xG_xG_2Ht.update_layout(
    title_text='Ht2: Expectedgoals Opponent - Expectedgoals', title_x=title_x,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))
figHistogramA_xG_xG_2Ht.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="right",
    x=1.12
))


st.title("Football statistics - {}".format(team))
st.markdown('The following two diagrams display the new metric Expected Goals (**xGoals**), which is a qualitative measurement on base of the shots on goal.  \nThe expected goal model shows how high the chance of the goal really was and calculates a value for each completion based on several factors.   \nF.I. a penalty has generally a probably of 75 % to result in a goal, which would increase the xGoal value for 0.75 regardless of the penalty-outcame in this case.', unsafe_allow_html=False)

col1, col2 = st.columns(2)

col1.plotly_chart(BarBallpossesionstylesResultsHalftime1)
col2.plotly_chart(BarBallpossesionstylesResultsHalftime2)
col1.plotly_chart(BarBallpossesionstylesXGHalftime1)
col2.plotly_chart(BarBallpossesionstylesXGHalftime2)

col1.plotly_chart(figHistogramxG_A_xG_1Ht)
col1.plotly_chart(figHistogramA_xG_xG_1Ht)
col2.plotly_chart(figHistogramxG_A_xG_2Ht)
col2.plotly_chart(figHistogramA_xG_xG_2Ht)

col1.plotly_chart(figScatter_SoG_SoGA)
col2.plotly_chart(figScatter_SoGA_soG)
col1.plotly_chart(figScatter_h1_soG_SoGA)
col1.plotly_chart(figScatter_h1_SoGA_soG)
col2.plotly_chart(figScatter_h2_SoG_SoGA)
col2.plotly_chart(figScatter_h2_SoGA_SoG)

st.plotly_chart(fig_xg_perminute_home)
st.plotly_chart(fig_xg_homexg_complete_game_all_bpse)
st.plotly_chart(fig_xg_perminute_home_bigger_55)
st.plotly_chart(fig_xg_perminute_home_smaller_45)

# C_WPercText, N_WPercText, BP_WPercText = calc_stats(df4Complete)
# col2.write("% W < 0.45:   {}   \n % W 0.45 - 0.55:  {}   \n % W > 0.55:  {}".format(
#     C_WPercText, N_WPercText, BP_WPercText))

# show df
st.dataframe(df4Complete_show.style.format({'xG': '{:.1f}', 'A_xG': '{:.1f}', 'SoG-H': '{:.0f}',
                                            'G-H': '{:.0f}', 'G-A': '{:.0f}', 'BP-H': '{:.0f}',
                                            'BP-A': '{:.0f}', 'GA-H': '{:.0f}', 'GA-A': '{:.0f}',
                                            'xPTS': '{:.1f}', 'A_xPTS': '{:.1f}', 'SoG-A': '{:.0f}',
                                            }))


# Erstelle ein Seitenleisten-Menü
page = st.sidebar.radio("Choose a page:", ( 'League Tables', 'Team Analyis'))

# Navigiere zur ausgewählten Seite
if page == 'Team Analyis':
    page_teamx()
elif page == 'League Tables':
    page_league_table()